In [1]:
# All the imports for this file
from crewai import (
    Agent, 
    Task, 
    Crew, 
    LLM
)
from crewai.tools import tool
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from groq import Groq
import torch
import os
import json

c:\Users\ACER\Documents\Personal_projects\meal_planner\meal_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize embedding model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Initialize Qdrant client
qdrant = QdrantClient(
    url=os.environ.get("QDRANT_URL"),
    api_key=os.environ.get("QDRANT_API_KEY")
)

# Vector Search function and tool for both RAG Systems
@tool("Vector Search Tool")
def vector_search(query: str) -> str:
    """Search recipes vector database using semantic embeddings."""
    assert isinstance(query, str), "Your search query must be a string"
    query_vector = embedding_model.encode(query).tolist()
    results = qdrant.search(
        collection_name="recipe_data",
        query_vector=query_vector,
        limit=5,
    )
    return "Retrieved recipes:\n\n".join([str(result.payload) for result in results])

# Seperatly defining this due to conflicts with the tool decorator
def rag(query: str) -> str:
    """Search recipes vector database using semantic embeddings."""
    assert isinstance(query, str), "Your search query must be a string"
    query_vector = embedding_model.encode(query).tolist()
    results = qdrant.search(
        collection_name="recipe_data",
        query_vector=query_vector,
        limit=5,
    )
    return "Retrieved recipes:\n\n".join([str(result.payload) for result in results])

# Configure Llama3.3 70B LLM for Agentic RAG
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Setup Groq Client for Standard RAG and the LLM judge
groq_llm = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Setting the role, goal and backstory of the agent
role = "You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools."
goal = """Task: 
The user will provide details about the ingredients they have, dietary restrictions, allergy information, and their daily protein target. 
Using the tools provided to you, create three distinct recipes-one for breakfast, one for lunch, and one for dinner-that the user will repeat daily for 7 days. Also suggest one recipe for a mid-day snack that they can use to get additional protein. 
Each recipe should include specific ingredient quantities for meal prepping for 7 servings, ensuring each serving meets or closely aligns with the user's daily protein target when divided across meals."""
backstory = """Instructions:
Step 1: Analyze the user's input to understand the ingredients they have, their dietary restrictions, allergies, and daily protein target.
Step 2: Using the "vector_search" tool, query the vector database appropriately to retrieve relevant recipes based on the user's input. If the tool does not return sufficient recipes or if the recipes do not match the user's requirements, generate custom recipes by combining similar techniques and ingredient profiles.
Step 3: Follow the guidelines stated below and create the meal plan for the user.

Guidelines: 
1. Dietary and Allergy Compliance: Ensure recipes strictly adhere to the user's dietary restrictions and allergies.  
2. Protein Target: Use the daily protein target provided by the user to guide recipe creation. Ensure the protein content of each serving from breakfast, lunch, and dinner combines to meet or is as close as possible to the target. Distribute the protein evenly across meals or based on meal size.
3. Ingredient Utilization: Maximize the use of the user-provided ingredients. If additional ingredients are required, include them in the ingredients list and tag them as "Must be purchased."
4. Nutritional Information: Provide the nutritional details (calories, protein, fats, and carbohydrates) for one serving of each meal. Do this only after the recipe is finalized.
5. Meal Prep Quantities: Clearly indicate the total quantity of each ingredient needed to prepare all 7 servings of the recipe.  
6. Preparation Instructions: Include concise, step-by-step instructions for preparing each recipe, suitable for batch cooking and storage.
7. Formatting: Present the meal plan in a clear, organized and human readable format including the name of the recipe, the total quantity of each ingredient needed to prepare all 7 servings of the recipe, the steps to cook each recipe and the nutritional information for each serving."""

In [3]:
# Meal planning agent
meal_planner_agent = Agent(
    role=role,
    goal=goal,
    backstory=backstory,
    llm=llm,
    tools=[vector_search],
    verbose=True
)

def agentic_rag_mp(user_input: str):
    """This tool creates a crew and gives them a task to create a meal plan for the user and returns the meal plan as the output."""
    assert isinstance(user_input, str), "User input must be a string"

    # Create the crew
    meal_planning_crew = Crew(
        agents=[meal_planner_agent],
        tasks=[],
        verbose=True
    )

    # Create the task
    meal_planning_task = Task(
        description=f"Create a complete weekly meal plan based on the user's input.\n\n{user_input}",
        expected_output="",
        agent=meal_planner_agent
    )

    # Add the task to the crew
    meal_planning_crew.tasks = [meal_planning_task]
    
    # Execute the crew,
    meal_plan = meal_planning_crew.kickoff()
    
    return meal_plan

In [4]:
# Creating the function to perform Standard RAG
def standard_rag_mp(user_input: str):
    rag_output = rag(user_input)
    prompt = f"""You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
    
Task: 
The user will provide details about the ingredients they have, dietary restrictions, allergy information, and their daily protein target. 
Using the tools provided to you, create three distinct recipes-one for breakfast, one for lunch, and one for dinner-that the user will repeat daily for 7 days. Also suggest one recipe for a mid-day snack that they can use to get additional protein. 
Each recipe should include specific ingredient quantities for meal prepping for 7 servings, ensuring each serving meets or closely aligns with the user's daily protein target when divided across meals.

Instructions:
Step 1: Analyze the user's input to understand the ingredients they have, their dietary restrictions, allergies, and daily protein target.
Step 2: Using the "vector_search" tool, query the vector database appropriately to retrieve relevant recipes based on the user's input. If the tool does not return sufficient recipes or if the recipes do not match the user's requirements, generate custom recipes by combining similar techniques and ingredient profiles.
Step 3: Follow the guidelines stated below and create the meal plan for the user.

Guidelines: 
1. Dietary and Allergy Compliance: Ensure recipes strictly adhere to the user's dietary restrictions and allergies.  
2. Protein Target: Use the daily protein target provided by the user to guide recipe creation. Ensure the protein content of each serving from breakfast, lunch, and dinner combines to meet or is as close as possible to the target. Distribute the protein evenly across meals or based on meal size.
3. Ingredient Utilization: Maximize the use of the user-provided ingredients. If additional ingredients are required, include them in the ingredients list and tag them as "Must be purchased."
4. Nutritional Information: Provide the nutritional details (calories, protein, fats, and carbohydrates) for one serving of each meal. Do this only after the recipe is finalized.
5. Meal Prep Quantities: Clearly indicate the total quantity of each ingredient needed to prepare all 7 servings of the recipe.  
6. Preparation Instructions: Include concise, step-by-step instructions for preparing each recipe, suitable for batch cooking and storage.
7. Formatting: Present the meal plan in a clear, organized and human readable format including the name of the recipe, the total quantity of each ingredient needed to prepare all 7 servings of the recipe, the steps to cook each recipe and the nutritional information for each serving.

User input: 

{user_input}

{rag_output}
"""
    chat_completion = groq_llm.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.3-70b-versatile",
    )
    return chat_completion.choices[0].message.content, rag_output

In [5]:
# Creating the LLM Judge
def llm_judge(user_input: str, agentic_rag_output: str, standard_rag_output: str):
    prompt = f"""You are an expert examiner with 30+ years of experience in evaluating answers based on the task description, inputs and marking guidelines.
Your goal will be to evaluate the responses of two RAG systems and determine which one is better based on the information provided to you and a given criteria and respond adhering to a strict output format provided.
You will be provided with the user input, the task which both systems executed, the documents retrieved by the vector search and the outputs both systems generated.

Their task: 
Create a meal plan consisting of a breakfast, lunch, dinner and snack recipe based on the ingredients, dietary restrictions, allergy information, and daily protein intake target provied. 
The recipes would be meal prepped and repeated by the user for 7 days hence the meal plan should include the total quantity of each ingredient needed to prepare all 7 servings of the recipe.
The recipes should contain the recipe name, ingredients and steps to cook the recipe. The meal plan should also include the calories, proteins, fats and carbohydrates for each recipe.
The recipes should hit the daily protein intake target as close as possible.
The recipes should utilize the ingredients provided by the user as much as possible and if any additional ingredients are required, they should be tagged as "Must be purchased".

Criteria:
Compare the outputs and score them based on Context Relevance, and Answer Accuracy.

1. Context Relevance: Examine the query used by each system to retrieve the documents from the vector database and evaluate how relevant the retrieved documents are to the user input and task. Score out of 10, where 1 is the least relevant and 10 is highly relevant.
2. Answer Accuracy: Assess how accurately each output satisfies the constraints given in the task, i.e., hitting or closely reaching the daily protein intake target, minimizing the usage of ingredients not listed in the user's ingredients list, and adhering to dietary restrictions and allergy information. Score out of 10, where 1 is highly inaccurate and 10 is highly accurate.

Output Format:

Agentic RAG System:
- Context Relevance score: [/10]
- Answer Accuracy score: [/10]
Feedback: [A brief summary of why you gave these scores]

Standard RAG System:
- Context Relevance score: [/10]
- Answer Accuracy score: [/10]
Feedback: [A brief summary of why you gave these scores]

Final Verdict: [Agentic RAG System performed better or Standard RAG System performed better]

Here's all the information you will need to evaluate the two systems:

User input:
{user_input}

Agentic RAG System Output including vector search query/queries and results:
{agentic_rag_output}

Standard RAG System Output including vector search query and results:
{standard_rag_output}"""

    chat_completion = groq_llm.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama-3.3-70b-versatile",
    )
    return chat_completion.choices[0].message.content

In [6]:
inputs = {
    "input1": 
        """Ingredients: Salmon fillets, shrimp, chicken thighs, quinoa, brown rice, zucchini, eggplant, bell peppers (red, yellow), tomatoes, cucumbers, olives (kalamata), artichoke hearts, spinach, romaine lettuce, red onion, garlic, lemon, olive oil, avocado, almonds, walnuts, pine nuts, basil, oregano, thyme, rosemary, parsley, capers, balsamic vinegar.
Dietary information: Dairy-Free, Gluten-Free
Allergy information: None
Daily protein target: 120g""",
    "input2": 
        """Ingredients: Chicken breast, salmon, eggs, tofu, brown rice, quinoa, oats, whole wheat bread, sweet potatoes, broccoli, spinach, carrots, bell peppers (red, green, yellow), onions, garlic, olive oil, avocado, almonds, walnuts, bananas, apples, oranges, berries (strawberries, blueberries, raspberries), Greek yogurt, milk, cheese
Dietary information: None
Allergy information: None
Daily protein target: 120g""",
    "input3": 
        """Ingredients: Tofu, tempeh, lentils, chickpeas, black beans, kidney beans, quinoa, brown rice, farro, oats, whole wheat pasta, spinach, kale, broccoli, cauliflower, carrots, bell peppers, mushrooms, onions, garlic, olive oil, coconut oil, avocado, almonds, walnuts, cashews, sunflower seeds, pumpkin seeds, peanut butter, soy milk, almond milk, nutritional yeast
Dietary information: Vegan
Allergy information: Peanut, Soy
Daily protein target: 100g""",
    "input4": 
        """Ingredients: Ground beef, chicken thighs, cod fillets, shrimp, eggs, brown rice, couscous, whole wheat tortillas, potatoes, sweet potatoes, green beans, asparagus, zucchini, tomatoes, onions, garlic, ginger, cilantro, parsley, basil, oregano, thyme, rosemary, cumin, coriander, turmeric, chili powder, paprika, olive oil, butter, coconut oil, avocado, walnuts, pecans, apples, pears, grapes, lemons, limes, honey
Dietary information: None
Allergy information: None
Daily protein target: 105g""",
    "input5": 
        """Ingredients: Salmon, tuna, sardines, eggs, quinoa, brown rice, wild rice, farro, whole wheat bread, rye bread, oats, avocados, olive oil, flaxseed oil, chia seeds, hemp seeds, sunflower seeds, almonds, walnuts, pecans, spinach, kale, broccoli, Brussels sprouts, carrots, sweet potatoes, onions, garlic, ginger, lemons, limes, berries (strawberries, blueberries, blackberries)
Dietary information: Pescatarian
Allergy information: None
Daily protein target: 95g""",
    "input6": 
        """Ingredients: Salmon, cod, shrimp, chicken breast, lamb, chickpeas, lentils, quinoa, couscous, whole wheat pasta, spinach, kale, romaine lettuce, tomatoes, cucumbers, bell peppers (red, green, yellow), onions, garlic, olives, olive oil, avocado, feta cheese, hummus, almonds, walnuts, pine nuts, lemons, oranges, grapes, figs, dates, oregano, thyme, rosemary, mint
Dietary information: None
Allergy information: None
Daily protein target: 85g""",
    "input7": 
        """Ingredients: Tofu, tempeh, chicken breast, shrimp, brown rice, jasmine rice, rice noodles, bok choy, broccoli, carrots, mushrooms, shiitake mushrooms, onions, garlic, ginger, soy sauce, sesame oil, peanut butter, rice vinegar, chili flakes, cilantro, scallions, peanuts, cashews
Dietary information: None
Allergy information: Peanut, Soy
Daily protein target: 75g""",
    "input8": 
        """Ingredients: Canned tuna, canned beans (kidney, black, chickpeas), lentils, rice, pasta, potatoes, onions, carrots, frozen vegetables (peas, corn, broccoli), eggs, peanut butter, oats, whole wheat bread, canned tomatoes, olive oil, spices (cumin, chili powder, oregano)
Dietary information: None
Allergy information: Peanut
Daily protein target: 100g""",
    "input9": 
        """Ingredients: Turkey breast, lean ground beef, tilapia fillets, egg whites, brown rice, wild rice, quinoa, steel-cut oats, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper.
Dietary information: None
Allergy information: None
Daily protein target: 120g""",
    "input10": 
        """Ingredients: Turkey breast, lean ground beef, tilapia fillets, egg whites, brown rice, wild rice, quinoa, steel-cut oats, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper.
Dietary information: None
Allergy information: None
Daily protein target: 95g"""
}

In [7]:
# Initializing the dictionary to store the outputs of the Standard RAG system and the outputs of the Agentic RAG system
standard_rag_outputs = {}
agentic_rag_outputs = {}

### Executing everything one by one with intervals to avoid rate limits

In [8]:
standard_mp, docs = standard_rag_mp(inputs["input1"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input1"]}

Output:
{docs}"""
standard_rag_outputs["input1"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [9]:
standard_mp, docs = standard_rag_mp(inputs["input2"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input2"]}

Output:
{docs}"""
standard_rag_outputs["input2"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [10]:
standard_mp, docs = standard_rag_mp(inputs["input3"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input3"]}

Output:
{docs}"""
standard_rag_outputs["input3"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [11]:
standard_mp, docs = standard_rag_mp(inputs["input4"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input4"]}

Output:
{docs}"""
standard_rag_outputs["input4"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [12]:
standard_mp, docs = standard_rag_mp(inputs["input5"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input5"]}

Output:
{docs}"""
standard_rag_outputs["input5"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [13]:
standard_mp, docs = standard_rag_mp(inputs["input6"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input6"]}

Output:
{docs}"""
standard_rag_outputs["input6"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [14]:
standard_mp, docs = standard_rag_mp(inputs["input7"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input7"]}

Output:
{docs}"""
standard_rag_outputs["input7"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [15]:
standard_mp, docs = standard_rag_mp(inputs["input8"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input8"]}

Output:
{docs}"""
standard_rag_outputs["input8"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [16]:
standard_mp, docs = standard_rag_mp(inputs["input9"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input9"]}

Output:
{docs}"""
standard_rag_outputs["input9"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [17]:
standard_mp, docs = standard_rag_mp(inputs["input10"])
final_output = f"""{standard_mp} 

Vector Seach Query:
{inputs["input10"]}

Output:
{docs}"""
standard_rag_outputs["input10"] = final_output

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:29: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(


In [18]:
# Printing the outputs of the Standard RAG system
print(json.dumps(standard_rag_outputs, indent=4))

{
    "input1": "### Meal Plan for 7 Days\n\nGiven the user's input and dietary requirements, I've created three distinct recipes for breakfast, lunch, and dinner, along with a mid-day snack. These recipes are designed to meet the daily protein target of 120g while adhering to the dairy-free and gluten-free dietary restrictions.\n\n#### Breakfast Recipe: Quinoa and Vegetable Bowl with Salmon\n\n**Ingredients for 7 servings:**\n- 1 cup quinoa (Must be purchased if not available)\n- 2 cups water\n- 2 salmon fillets, cooked and diced\n- 4 cups mixed vegetables (zucchini, eggplant, bell peppers)\n- 1/4 cup olive oil\n- 2 cloves garlic, minced\n- 1 tablespoon lemon juice\n- Salt and pepper to taste\n- Fresh parsley, chopped (for garnish)\n\n**Steps:**\n1. Cook quinoa according to package instructions using 2 cups of water.\n2. Grill or bake the salmon fillets until cooked, then dice.\n3. Saut\u00e9 the mixed vegetables and garlic in olive oil until tender.\n4. Combine cooked quinoa, diced s

### Executing everything one by one to avoid rate limit issues and to extract the tool input and output

In [19]:
print(agentic_rag_mp(inputs["input1"]))

# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Salmon fillets, shrimp, chicken thighs, quinoa, brown rice, zucchini, eggplant, bell peppers (red, yellow), tomatoes, cucumbers, olives (kalamata), artichoke hearts, spinach, romaine lettuce, red onion, garlic, lemon, olive oil, avocado, almonds, walnuts, pine nuts, basil, oregano, thyme, rosemary, parsley, capers, balsamic vinegar.
Dietary information: Dairy-Free, Gluten-Free
Allergy information: None
Daily protein target: 120g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: To create a complete weekly meal plan based on the user's input, I need to first analyze the ingredients they have, their dietary restrictions, allergies, and daily protein target. The user has provided a list of ingredients, including salmon fillets, shrimp, chicken thighs, quinoa, brown rice, various vegetables, nuts, and herbs. They have also specified that they follow a dairy-free and gluten-free diet. The daily protein target is 120g.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"gluten-free dairy-free high protein recipes using salmon, shrimp, chicken, quinoa, brown rice, vegetables, nuts, and herbs\"}"
## Tool Output: 
{'source_column': 'ingredients', 'text': '1 can salmon, 1 onion, chopped up fine, 1 egg, 1 c. grated cheese, 1 Tbsp. butter, 1/4 c. milk, corn meal an

In [20]:
agentic_rag_outputs["input1"] = """# Weekly Meal Plan

## Breakfast: Grilled Chicken and Vegetable Skillet
### Ingredients:
- 7 cups mixed vegetables (bell peppers, onions, zucchini)
- 7 cups cooked chicken thighs
- 3.5 cups cooked quinoa
- 7 tablespoons olive oil
- Salt and pepper to taste
- Fresh parsley, chopped (optional)
### Instructions:
1. Preheat the grill or grill pan to medium-high heat.
2. Season the chicken thighs with salt and pepper.
3. Grill the chicken for 5-6 minutes per side or until cooked through.
4. Meanwhile, heat the olive oil in a large skillet over medium heat.
5. Add the mixed vegetables and cook until tender, about 5 minutes.
6. Serve the grilled chicken with the vegetable skillet and quinoa.
### Nutritional Information (per serving):
- Calories: 420
- Protein: 36g
- Fats: 20g
- Carbohydrates: 20g

## Lunch: Baked Salmon with Roasted Vegetables
### Ingredients:
- 7 salmon fillets (6 oz each)
- 7 cups mixed vegetables (eggplant, tomatoes, red onion)
- 1.75 cups chopped fresh parsley
- 14 tablespoons olive oil
- Salt and pepper to taste
- Fresh lemon wedges (optional)
### Instructions:
1. Preheat the oven to 400°F (200°C).
2. Season the salmon fillets with salt and pepper.
3. Place the salmon on a baking sheet lined with parchment paper.
4. Roast the mixed vegetables in the oven for 15-20 minutes or until tender.
5. Serve the baked salmon with the roasted vegetables and a squeeze of lemon juice.
### Nutritional Information (per serving):
- Calories: 440
- Protein: 42g
- Fats: 24g
- Carbohydrates: 10g

## Dinner: Shrimp and Vegetable Stir-Fry
### Ingredients:
- 7 cups cooked shrimp
- 7 cups mixed vegetables (zucchini, bell peppers, onions)
- 3.5 cups cooked brown rice
- 7 tablespoons olive oil
- 7 teaspoons soy sauce (make sure it's gluten-free)
- Salt and pepper to taste
- Fresh basil, chopped (optional)
### Instructions:
1. Heat the olive oil in a large skillet or wok over medium-high heat.
2. Add the mixed vegetables and cook until tender, about 5 minutes.
3. Add the cooked shrimp and stir-fry for 2-3 minutes.
4. Serve the shrimp and vegetable stir-fry over brown rice.
### Nutritional Information (per serving):
- Calories: 400
- Protein: 34g
- Fats: 16g
- Carbohydrates: 30g

## Mid-day Snack: Avocado and Almond Snack
### Ingredients:
- 7 medium avocados
- 7 oz almonds
- Salt and pepper to taste
- Fresh parsley, chopped (optional)
### Instructions:
1. Cut the avocado in half and remove the pit.
2. Sprinkle salt and pepper to taste.
3. Serve with almonds and a sprinkle of parsley.
### Nutritional Information (per serving):
- Calories: 170
- Protein: 9g
- Fats: 14g
- Carbohydrates: 8g

This meal plan provides approximately 121g of protein per day, meeting the user's daily protein target. The recipes are all dairy-free and gluten-free, and the ingredients are maximized to reduce food waste. The meal plan includes a variety of vegetables, lean proteins, and healthy fats to provide a balanced diet.

Vector Search Query:
"gluten-free dairy-free high protein recipes using salmon, shrimp, chicken, quinoa, brown rice, vegetables, nuts, and herbs"

Output: 
{'source_column': 'ingredients', 'text': '1 can salmon, 1 onion, chopped up fine, 1 egg, 1 c. grated cheese, 1 Tbsp. butter, 1/4 c. milk, corn meal and flour', 'title': 'Salmon Patties', 'ingredients': '1 can salmon, 1 onion, chopped up fine, 1 egg, 1 c. grated cheese, 1 Tbsp. butter, 1/4 c. milk, corn meal and flour', 'directions': 'Open the can of salmon and remove the bones., Add onion, egg, grated cheese, butter, milk and equal amounts of flour and corn meal to thicken the mixture so it will stick together., Put small amount of cooking oil in a skillet and spoon out patties and cook until golden brown.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 can pink salmon, 1 egg, beaten, 3 Tbsp. milk, 1 Tbsp. flour, 1 Tbsp. corn meal, chopped onion (optional)', 'title': 'Salmon Croquettes', 'ingredients': '1 can pink salmon, 1 egg, beaten, 3 Tbsp. milk, 1 Tbsp. flour, 1 Tbsp. corn meal, chopped onion (optional)', 'directions': 'Combine all ingredients., Spoon mixture into hot fat and fry until brown.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'eggs, Parmesan cheese, soy sauce, venison, cracker meal', 'title': "Pop'S Vinasnitzel(Venison)  ", 'ingredients': 'eggs, Parmesan cheese, soy sauce, venison, cracker meal', 'directions': 'For each egg, add 2 tablespoons Parmesan cheese and a dash of soy sauce., Mix gently., Cut meat into 2-inch squares, beat thin, dredge in egg mixture and roll in cracker meal., Fry over medium heat.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '6 oz. cooked yolk-free fine noodles, 1 1/2 c. (12 oz) 1 small curd cottage cheese, 1 pkg. (10 oz) frozen mixed vegetables, thawed and drained, 1 c. cubed fully cooked lean ham, 3/4 c. reduced-fat sour cream, 1/4 c. fat-free milk, 3 Tbsp. grated Parmesan cheese, 2 tsp. all purpose flour, 1 tsp. dill weed or 1 Tbsp snipped fresh dill, 1/4 tsp. salt', 'title': 'Ham And Noodle Casserole', 'ingredients': '6 oz. cooked yolk-free fine noodles, 1 1/2 c. (12 oz) 1% small curd cottage cheese, 1 pkg. (10 oz) frozen mixed vegetables, thawed and drained, 1 c. cubed fully cooked lean ham, 3/4 c. reduced-fat sour cream, 1/4 c. fat-free milk, 3 Tbsp. grated Parmesan cheese, 2 tsp. all purpose flour, 1 tsp. dill weed or 1 Tbsp snipped fresh dill, 1/4 tsp. salt', 'directions': 'Cook noodles according to package directions, drain., In a large bowl, combine the remaining ingredients., Add noodles and toss to coat., Transfer to a 2 qt. baking dish coated with nonstick cooking spray. Cover and bake 350 for 30 minutes. Uncover; bake 5 - 10 minutes longer or until heated through., Let stand for 5 minutes before serving., Yield: 4 servings.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 c. whole wheat flour, 2/3 c. sharp Cheddar, shredded, 6 Tbsp. vegetable oil, 1/4 c. chopped almonds, 1/4 tsp. salt, 1/4 tsp. paprika, 3 eggs, beaten, 1 c. sour cream, 1/4 c. mayonnaise, 1/2 c. shredded sharp Cheddar, 1 Tbsp. grated onion, 1/4 tsp. dill weed, 3 drops Tabasco sauce, 1 (15 1/2 oz.) can salmon', 'title': 'Salmon Quiche', 'ingredients': '1 c. whole wheat flour, 2/3 c. sharp Cheddar, shredded, 6 Tbsp. vegetable oil, 1/4 c. chopped almonds, 1/4 tsp. salt, 1/4 tsp. paprika, 3 eggs, beaten, 1 c. sour cream, 1/4 c. mayonnaise, 1/2 c. shredded sharp Cheddar, 1 Tbsp. grated onion, 1/4 tsp. dill weed, 3 drops Tabasco sauce, 1 (15 1/2 oz.) can salmon', 'directions': 'Combine the first 6 ingredients; reserve 1/2 cup and press the rest into a 9-inch quiche dish, bottom and sides., Bake at 400 for 10 minutes and set aside., Reduce oven to 325.'}"""

In [21]:
print(agentic_rag_mp(inputs["input2"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Chicken breast, salmon, eggs, tofu, brown rice, quinoa, oats, whole wheat bread, sweet potatoes, broccoli, spinach, carrots, bell peppers (red, green, yellow), onions, garlic, olive oil, avocado, almonds, walnuts, bananas, apples, oranges, berries (strawberries, blueberries, raspberries), Greek yogurt, milk, cheese
Dietary information: None
Allergy information: None
Daily protein target: 120g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: To create a comprehensive weekly meal plan for the user, I first need to analyze the provided ingredients, dietary restrictions, allergies, and the daily protein target. The user has a varied list of ingredients including proteins like chicken breast, salmon, eggs, tofu, and Greek yogurt, which can be used to meet the daily protein target of 120g. Since there are no dietary restrictions or allergies, I can plan the meals without any specific limitations. 
The next step involves using the Vector Search Tool to find suitable recipes that incorporate the given ingredients and meet the protein target. I will start by querying the Vector Search Tool for breakfast recipes that include high-protein ingredients like eggs, chicken breast, or Greek yogurt.
## Using tool: Vector Search Tool
## Tool Inpu

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Retrieved recipes:
Let's choose the Breakfast Eggs Plus recipe as a starting point for the breakfast meal. However, since it does not include the user's ingredients exactly and lacks nutritional information, we will modify it to better suit the user's needs.
Thought: Now that we have a starting point for breakfast, we need to adjust the recipe to incorporate the user's ingredients and meet the daily protein target. We will use chicken breast, eggs, and Greek yogurt to create a high-protein breakfast recipe.
Modified Breakfast Recipe:
- 2 lbs chicken breast, diced
- 1 dozen eggs
- 1 cup Greek yogurt
- 1 cup mixed bell peppers (red, green, yellow)
- 1 cup broccoli
- 1/2 cup shredded cheese
- 1/4 cup chopped almonds
- 1 tablespoon olive oil
- Salt and pepper to taste
Instructions:
1. Preheat the oven to 

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Retrieved recipes:
Thought: Now that we have some ideas for salmon quiche, let's modify the recipe to include quinoa and meet the user's protein target.
Modified Lunch Recipe:
- 1 lb salmon fillet, cooked and flaked
- 1 cup cooked quinoa
- 1 cup mixed broccoli and carrots
- 1/2 cup Greek yogurt
- 1/4 cup chopped walnuts
- 1 tablespoon olive oil
- Salt and pepper to taste
Instructions:
1. Preheat the oven to 375°F.
2. In a large bowl, mix the cooked quinoa, mixed broccoli and carrots, and chopped walnuts.
3. In a separate bowl, whisk the Greek yogurt and set aside.
4. Heat the olive oil in a large skillet over medium heat. Add the cooked salmon and stir well.
5. Combine the quinoa mixture and salmon, then top with Greek yogurt.
6. Serve hot and enjoy!
Nutritional Information (per serving):
- Calories: 

In [22]:
agentic_rag_outputs["input2"] = """Here is your complete meal plan for the week:

**Monday to Sunday**

* **Breakfast**: 
  - 2 lbs chicken breast, diced
  - 1 dozen eggs
  - 1 cup Greek yogurt
  - 1 cup mixed bell peppers (red, green, yellow)
  - 1 cup broccoli
  - 1/2 cup shredded cheese
  - 1/4 cup chopped almonds
  - 1 tablespoon olive oil
  - Salt and pepper to taste

  Instructions:
  1. Preheat the oven to 375°F.
  2. In a large bowl, whisk the eggs and set aside.
  3. In a separate bowl, mix the diced chicken breast with Greek yogurt, salt, and pepper.
  4. Heat the olive oil in a large skillet over medium heat. Add the mixed bell peppers and broccoli and cook until tender.
  5. Add the cooked chicken and egg mixture to the skillet and stir well.
  6. Pour the egg and chicken mixture into a greased 9x13-inch baking dish.
  7. Top with shredded cheese and chopped almonds.
  8. Bake for 35-40 minutes or until the eggs are set.

  Nutritional Information (per serving):
  - Calories: 320
  - Protein: 37g
  - Fats: 12g
  - Carbohydrates: 20g

* **Lunch**: 
  - 1 lb salmon fillet, cooked and flaked
  - 1 cup cooked quinoa
  - 1 cup mixed broccoli and carrots
  - 1/2 cup Greek yogurt
  - 1/4 cup chopped walnuts
  - 1 tablespoon olive oil
  - Salt and pepper to taste

  Instructions:
  1. Preheat the oven to 375°F.
  2. In a large bowl, mix the cooked quinoa, mixed broccoli and carrots, and chopped walnuts.
  3. In a separate bowl, whisk the Greek yogurt and set aside.
  4. Heat the olive oil in a large skillet over medium heat. Add the cooked salmon and stir well.
  5. Combine the quinoa mixture and salmon, then top with Greek yogurt.
  6. Serve hot and enjoy!

  Nutritional Information (per serving):
  - Calories: 400
  - Protein: 40g
  - Fats: 15g
  - Carbohydrates: 30g

* **Dinner**: 
  - 1 lb chicken breast, diced
  - 2 large sweet potatoes, peeled and diced
  - 1 cup mixed broccoli and carrots
  - 1/4 cup chopped almonds
  - 1 tablespoon olive oil
  - Salt and pepper to taste

  Instructions:
  1. Preheat the oven to 400°F.
  2. In a large bowl, toss the diced sweet potatoes with olive oil, salt, and pepper.
  3. Spread the sweet potatoes on a baking sheet and roast for 20-25 minutes or until tender.
  4. In a large skillet, heat the olive oil over medium heat. Add the diced chicken breast and cook until browned.
  5. Add the mixed broccoli and carrots to the skillet and cook until tender.
  6. Serve the chicken and vegetable mixture over the roasted sweet potatoes.

  Nutritional Information (per serving):
  - Calories: 420
  - Protein: 43g
  - Fats: 10g
  - Carbohydrates: 35g

* **Mid-day Snack**: 
  - 1 cup Greek yogurt
  - 1/2 cup mixed berries (strawberries, blueberries, raspberries)
  - 1/4 cup chopped almonds
  - 1 tablespoon honey

  Instructions:
  1. In a small bowl, mix the Greek yogurt and honey.
  2. Top the yogurt with mixed berries and chopped almonds.
  3. Serve chilled and enjoy!

  Nutritional Information (per serving):
  - Calories: 200
  - Protein: 20g
  - Fats: 8g
  - Carbohydrates: 25g

Remember to drink plenty of water throughout the day and adjust the portion sizes based on your individual calorie needs. Enjoy your meals!

Vector Search Query 1:
"high protein breakfast recipes with eggs, chicken breast, or Greek yogurt"

Output: 
{'source_column': 'title', 'text': 'Breakfast Eggs Plus', 'title': 'Breakfast Eggs Plus', 'ingredients': '1 doz. eggs, 2 lb. Cheddar cheese, 1 lb. bacon, 1 (1 lb.) pkg. frozen broccoli, 1/2 loaf white bread', 'directions': 'Grease cake pan (9 x 13-inches); whip eggs and set aside. Cube bread enough to cover bottom of pan with a good layer., Dump on broccoli and cooked and drained bacon and egg mixture., Top with cheese., Bake for 1 hour at 350.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Breakfast Eggs', 'title': 'Breakfast Eggs', 'ingredients': '8 eggs, slightly beaten, 6 slices bread including crusts, cubed, 1 c. sharp Cheddar cheese, grated, 1 lb. sausage, cooked and crumbled, 2 c. milk, 1 tsp. salt, 1 tsp. dry mustard', 'directions': 'Combine ingredients., Place in greased 9 x 13-inch pan. Refrigerate 12 hours., Bake 35 minutes at 350.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Breakfast Egg Dish', 'title': 'Breakfast Egg Dish', 'ingredients': '1 lb. sweet sausage, cooked and crumbled, 12 slices buttered bread, crust removed, 10 oz. grated Cheddar cheese, 2 c. milk, 6 eggs', 'directions': 'Line a 9 x 13-inch glass pan with buttered bread, buttered side down., Sprinkle cooked sausage on top., Mix eggs and milk together well; pour on next., Sprinkle cheese over top., Set overnight., Bake at 350 for 45 minutes, covered, then 15 minutes, uncovered.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 1/4 c. plain low-fat yogurt, 1 large hard-cooked egg, peeled and chopped, 3 Tbsp. ketchup, 2 Tbsp. finely chopped dill pickle, few grains black pepper', 'title': 'Low Calorie Island Dressing', 'ingredients': '1 1/4 c. plain low-fat yogurt, 1 large hard-cooked egg, peeled and chopped, 3 Tbsp. ketchup, 2 Tbsp. finely chopped dill pickle, few grains black pepper', 'directions': 'Mix all ingredients in a bowl or screw-top jar., Makes 1 1/2 cups dressing., Keeps a week in refrigerator., Two tablespoons is 26 calories.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '2 lb. whole chicken, cut up, 1 c. unflavored low-fat yogurt, 1 1/2 c. regular wheat germ, 1/2 c. finely chopped almonds, 1/2 tsp. salt, 1/4 tsp. pepper, 1 tsp. dried dill weed', 'title': 'Dill Chicken', 'ingredients': '2 lb. whole chicken, cut up, 1 c. unflavored low-fat yogurt, 1 1/2 c. regular wheat germ, 1/2 c. finely chopped almonds, 1/2 tsp. salt, 1/4 tsp. pepper, 1 tsp. dried dill weed', 'directions': 'Mix together wheat germ, almonds, salt, pepper and dill weed. Dip chicken pieces in yogurt and then wheat germ mixture; dredge to coat.'}

Vector Search Query 2:
"salmon and quinoa recipes high protein"

Output: 
{'source_column': 'title', 'text': 'Salmon Quiche', 'title': 'Salmon Quiche', 'ingredients': '1 c. whole wheat flour, 2/3 c. sharp Cheddar, shredded, 6 Tbsp. vegetable oil, 1/4 c. chopped almonds, 1/4 tsp. salt, 1/4 tsp. paprika, 3 eggs, beaten, 1 c. sour cream, 1/4 c. mayonnaise, 1/2 c. shredded sharp Cheddar, 1 Tbsp. grated onion, 1/4 tsp. dill weed, 3 drops Tabasco sauce, 1 (15 1/2 oz.) can salmon', 'directions': 'Combine the first 6 ingredients; reserve 1/2 cup and press the rest into a 9-inch quiche dish, bottom and sides., Bake at 400 for 10 minutes and set aside., Reduce oven to 325.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Salmon Quiche', 'title': 'Salmon Quiche', 'ingredients': '1 prebaked (9-inch) deep dish pie shell, 1 Tbsp. butter or margarine, 1/4 c. minced onion, 1/4 c. minced green pepper, 1 can (7 3/4 oz.) salmon, well drained or 1 c. cooked salmon, 1 1/2 c. shredded Monterey Jack cheese, 4 eggs, 1 1/4 c. evaporated milk, 1/4 tsp. salt, dash of pepper, 1/4 tsp. dill weed, 2 Tbsp. chopped green onion', 'directions': 'In small skillet, melt butter; add onion, celery and green pepper., Saute until onion is translucent., Arrange vegetable mixture evenly in bottom of baked pie shell., Crumble salmon in an even layer over vegetables; sprinkle evenly with cheese.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Salmon Quiche', 'title': 'Salmon Quiche', 'ingredients': '1 c. whole wheat flour, 2/3 c. shredded sharp Cheddar cheese, 1/4 c. chopped almonds, 1/4 tsp. salt, 1/4 tsp. paprika, 1/4 c. plus 2 Tbsp. vegetable oil, 3 eggs, beaten, 1 (8 oz.) carton commercial sour cream, 1/4 c. mayonnaise, 1/2 c. (2 oz.) shredded Cheddar cheese, 1 Tbsp. grated onion, 1/4 tsp. dried whole dill weed, 3 drops hot sauce, 1 (15 1/2 oz.) can salmon (undrained), celery leaves (optional), cherry tomato (optional)', 'directions': 'Combine flour and next 5 ingredients; mix well., Reserve 1/2 cup crust mixture and set aside. Press mixture into bottom and up sides of 9-inch quiche dish., Bake at 400 for 10 minutes., Reduce heat to 325., Remove crust from oven and set aside.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Salmon Quiche', 'title': 'Salmon Quiche', 'ingredients': '1 (9-inch) pie crust, 1 (15 1/2 oz.) can salmon, 3 eggs, 1 c. sour cream, 1/4 c. mayo, 1/2 c. shredded sharp Cheddar cheese, 1 Tbsp. grated onion', 'directions': 'Flake salmon., In a bowl, blend together eggs, sour cream, mayo and reserved salmon liquid., Stir in salmon, onion and cheese., Spoon filling into crust., Bake in a 325 oven for 45 minutes or until firm.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Salmon Quiche', 'title': 'Salmon Quiche', 'ingredients': '1 (9 oz.) partially baked pastry shell, 1 small can red salmon, drained, 2 Tbsp. minced onions, 2 Tbsp. butter or margarine, 1 (3 oz.) can B & B mushrooms, drained (reserve broth), 4 eggs, beaten, 3/4 c. cream, 1/2 tsp. salt and pepper, 1/8 tsp. tarragon, 1/4 tsp. marjoram, 1/2 c. grated Swiss cheese', 'directions': 'Saute onions in butter until tender., Add mushrooms, reserving a few slices., Saute for a few more minutes.'}

Vector Search Query 3:
"chicken breast and sweet potato recipes high protein"

Output: 
{'source_column': 'title', 'text': 'Honey-Baked Chicken And Sweet Potatoes', 'title': 'Honey-Baked Chicken And Sweet Potatoes', 'ingredients': '1 chicken, in eighths, dash of salt, dash of pepper, 1/4 c. honey, 1/4 c. French dressing, 1 lb. can sweet potatoes', 'directions': 'Set oven to 375., Brush chicken with sauce mix (dressing, honey, salt and pepper) and bake, skin side down, for 30 minutes. Turn chicken and rebrush with sauce., Add sweet potatoes., Bake 15 minutes more.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Chicken Breast - Sauteed With New Potatoes', 'title': 'Chicken Breast - Sauteed With New Potatoes', 'ingredients': '1/4 c. margarine, 4 to 6 large chicken breasts, 1 slice onion, 1 minced clove garlic, 2 Tbsp. flour, 1/2 tsp. salt, 1/4 tsp. pepper, 1 chicken bouillon cube, 1 c. hot water, 1 (No. 303) can small potatoes, drained, 1/4 c. red wine, snipped parsley', 'directions': 'Saute chicken in large skillet in hot butter until both sides are brown., Add onion and garlic., Cook about 5 minutes., In a small bowl combine flour, salt and pepper and slowly stir in bouillon cube which has been dissolved in the hot water., Pour over browned chicken., Cook slowly covered for about 25 minutes or until chicken is tender., Add potatoes and wine., Heat potatoes thoroughly., Garnish with parsley., Yields 4 to 6 servings.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Low-Fat Sweet And Sour Chicken', 'title': 'Low-Fat Sweet And Sour Chicken', 'ingredients': 'cooked low-fat chicken nuggets, 1 large onion, 1 large green pepper, 2 large carrots, 1 can (small or medium) pineapple chunks (in juice), 3 sticks celery, 6 to 8 mushrooms, 1 c. water, 1 Tbsp. cornstarch, 1/2 c. vinegar, 1/2 c. sugar or sweetener, 1/2 c. tomato catsup, 1/4 c. soy sauce', 'directions': 'Cut vegetables into large chunks., Place with water in wok or large frying pan. Simmer until slightly crunchy. Combine cornstarch, vinegar, sugar, catsup and soy sauce., Add to vegetables. Cook until thickened., Add chicken nuggets and heat through., Serve over cooked rice or Chinese noodles.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '3 sweet potatoes or yams, boiled until tender, 1/3 c. butter, 1/4 c. brown sugar, 1 egg, well beaten, 2/3 c. milk, 1/2 tsp. vanilla, 1/2 tsp. nutmeg', 'title': 'Sweet Potato Pudding', 'ingredients': '3 sweet potatoes or yams, boiled until tender, 1/3 c. butter, 1/4 c. brown sugar, 1 egg, well beaten, 2/3 c. milk, 1/2 tsp. vanilla, 1/2 tsp. nutmeg', 'directions': 'Preheat oven to 350., Peel and mash potatoes., Combine butter and sugar and beat until creamy., Add to mashed potatoes and stir until well mixed., Slowly add beaten egg, milk and vanilla., Beat until well mixed., Sprinkle in nutmeg., Bake at 350 for 45 minutes.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '2 lb. boneless chicken breasts, 1 Tbsp. cornstarch, 1 Tbsp. soy sauce, 1/4 c. water, carrots, sliced, green pepper, sliced, mushrooms, sliced, water chestnuts, sliced, 1 (11.5 oz.) bottle Kikkoman sweet and sour sauce, 1 c. pineapple chunks', 'title': 'Sweet And Sour Chicken', 'ingredients': '2 lb. boneless chicken breasts, 1 Tbsp. cornstarch, 1 Tbsp. soy sauce, 1/4 c. water, carrots, sliced, green pepper, sliced, mushrooms, sliced, water chestnuts, sliced, 1 (11.5 oz.) bottle Kikkoman sweet and sour sauce, 1 c. pineapple chunks', 'directions': 'Cube chicken and coat with mixture of cornstarch and soy sauce., Stir fry in 2 tablespoons oil until brown, then add water and simmer 5 minutes., Add vegetables and sweet and sour sauce. Simmer uncovered 10 minutes., Take off heat and stir in pineapple. Serve over rice.'}"""

In [24]:
print(agentic_rag_mp(inputs["input3"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Tofu, tempeh, lentils, chickpeas, black beans, kidney beans, quinoa, brown rice, farro, oats, whole wheat pasta, spinach, kale, broccoli, cauliflower, carrots, bell peppers, mushrooms, onions, garlic, olive oil, coconut oil, avocado, almonds, walnuts, cashews, sunflower seeds, pumpkin seeds, peanut butter, soy milk, almond milk, nutritional yeast
Dietary information: Vegan
Allergy information: Peanut, Soy
Daily protein target: 100g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: Given the user's input, I need to create a weekly meal plan that is vegan, excludes peanuts and soy, and meets the daily protein target of 100g. Since the user has a soy allergy, I must exclude tofu, tempeh, and soy milk from the meal plan. I will use the Vector Search Tool to find recipes that incorporate the remaining ingredients and meet the user's requirements.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"vegan recipes with lentils, chickpeas, black beans, kidney beans, quinoa, brown rice, farro, oats, whole wheat pasta, spinach, kale, broccoli, cauliflower, carrots, bell peppers, mushrooms, onions, garlic, olive oil, coconut oil, avocado, almonds, walnuts, cashews, sunflower seeds, pumpkin seeds, almond milk, nutritional yeast, excluding peanuts and soy\"}"
## Tool Ou

In [25]:
agentic_rag_outputs["input3"] = """**Breakfast Recipe: Lentil and Quinoa Bowl**

- 4 cups quinoa
- 8 cups water
- 2 cups lentils
- 4 cups water
- 2 onions, diced
- 4 cloves garlic, minced
- 2 cups broccoli florets
- 2 cups carrots, peeled and grated
- 4 tbsp olive oil
- Salt and pepper, to taste

Steps:
1. In a medium saucepan, bring quinoa and water to a boil. Reduce heat and simmer for 15-20 minutes or until quinoa is tender.
2. In another saucepan, combine lentils and water. Bring to a boil, then reduce heat and simmer for 20-25 minutes or until lentils are tender.
3. In a large skillet, heat olive oil over medium heat. Add onion and garlic and cook until onion is translucent.
4. Add broccoli and carrots to the skillet and cook until tender.
5. To assemble the bowls, divide cooked quinoa and lentils between 7 containers. Top with the vegetable mixture and season with salt and pepper.

Nutritional information (per serving):
- Calories: 600
- Protein: 30g
- Fats: 15g
- Carbohydrates: 90g

**Lunch Recipe: Chickpea and Avocado Salad**

- 4 cans chickpeas, drained and rinsed
- 6 ripe avocados, diced
- 2 cups mixed greens
- 2 cups cherry tomatoes, halved
- 1 cup sliced red onion
- 1/2 cup chopped fresh parsley
- 4 tbsp olive oil
- 2 tbsp lemon juice
- Salt and pepper, to taste

Steps:
1. In a large bowl, combine chickpeas, avocado, mixed greens, cherry tomatoes, and red onion.
2. In a small bowl, whisk together olive oil and lemon juice.
3. Pour the dressing over the salad and toss to combine.
4. Sprinkle parsley over the top and season with salt and pepper.

Nutritional information (per serving):
- Calories: 650
- Protein: 30g
- Fats: 30g
- Carbohydrates: 60g

**Dinner Recipe: Black Bean and Sweet Potato Enchiladas**

- 8 cups cooked black beans
- 4 large sweet potatoes, peeled and diced
- 2 onions, diced
- 4 cloves garlic, minced
- 2 cups mixed greens
- 2 cups cherry tomatoes, halved
- 12 whole wheat tortillas
- 2 cups enchilada sauce
- 1/2 cup chopped fresh cilantro
- 2 tbsp olive oil
- Salt and pepper, to taste

Steps:
1. Preheat oven to 400°F (200°C).
2. In a large bowl, combine black beans, sweet potatoes, onion, and garlic.
3. In a separate bowl, combine mixed greens and cherry tomatoes.
4. To assemble the enchiladas, lay a tortilla flat and spoon some of the black bean mixture onto the center of the tortilla.
5. Roll up the tortilla and place seam-side down in a baking dish. Repeat with remaining tortillas and filling.
6. Pour enchilada sauce over the top of the enchiladas and cover with foil.
7. Bake for 25-30 minutes or until tortillas are tender.
8. Sprinkle cilantro over the top and serve with a side of mixed greens.

Nutritional information (per serving):
- Calories: 700
- Protein: 35g
- Fats: 20g
- Carbohydrates: 100g

**Mid-day Snack Recipe: Apple and Almond Butter**

- 14 apples, sliced
- 1 cup almond butter

Steps:
1. Spread 1-2 tbsp of almond butter onto each apple slice.

Nutritional information (per serving):
- Calories: 200
- Protein: 8g
- Fats: 16g
- Carbohydrates: 30g

Total daily protein intake: 103g (breakfast: 30g, lunch: 30g, dinner: 35g, snack: 8g)

Note: The ingredients listed are for 7 servings of each recipe. You can adjust the quantities based on your personal preferences and dietary needs. Also, make sure to drink plenty of water throughout the day to stay hydrated.

Vector Search Query:
"vegan recipes with lentils, chickpeas, black beans, kidney beans, quinoa, brown rice, farro, oats, whole wheat pasta, spinach, kale, broccoli, cauliflower, carrots, bell peppers, mushrooms, onions, garlic, olive oil, coconut oil, avocado, almonds, walnuts, cashews, sunflower seeds, pumpkin seeds, almond milk, nutritional yeast, excluding peanuts and soy\"}"

Output: 
{'source_column': 'ingredients', 'text': '1 can vegetarian red beans, 1 medium onion, 2 Tbsp. oil, 1/2 c. picante sauce, 1/2 c. shredded Monterey Jack cheese, 1/2 c. shredded Cheddar cheese, 1/3 c. sour cream, 1 bag corn tortilla chips, 1 bell pepper, 3 tsp. taco seasoning, 1 tsp. paprika', 'title': 'Mexican Bean Dip', 'ingredients': '1 can vegetarian red beans, 1 medium onion, 2 Tbsp. oil, 1/2 c. picante sauce, 1/2 c. shredded Monterey Jack cheese, 1/2 c. shredded Cheddar cheese, 1/3 c. sour cream, 1 bag corn tortilla chips, 1 bell pepper, 3 tsp. taco seasoning, 1 tsp. paprika', 'directions': 'In medium saucepan, heat oil., Saute bell pepper and onion for 2 minutes., Lower heat and add paprika and taco seasoning., Stir well., Add can of beans and mash with potato masher., Add 1/3 cup water., Cover and simmer for 8 minutes on low heat., Add picante sauce and serve in shallow round bowl., Top with both cheeses and heat in microwave for 3 minutes., Top with sour cream and serve with chips.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 box vegetarian burgers, 1 small onion, garlic powder, Italian seasoning, black pepper, vegetable oil, green pepper (optional), cheese of choice', 'title': 'Vegetarian Cheese Steak', 'ingredients': '1 box vegetarian burgers, 1 small onion, garlic powder, Italian seasoning, black pepper, vegetable oil, green pepper (optional), cheese of choice', 'directions': 'In a frying pan, slowly cook 4 patties of the vegetarian burgers in 2 tablespoons of vegetable oil until burgers are soft. Using a knife and fork, cut the burgers into small pieces.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'broccoli, peas, pea pods, green beans, green, yellow and red peppers, celery, carrots, bamboo shoots, water chestnuts, tofu (optional), cooked, chopped chicken (optional), 1/4 c. sesame seed oil, 1/4 c. teriyaki, 1 Tbsp. finely grated ginger, 1 large onion', 'title': 'My Favorite Dinner', 'ingredients': 'broccoli, peas, pea pods, green beans, green, yellow and red peppers, celery, carrots, bamboo shoots, water chestnuts, tofu (optional), cooked, chopped chicken (optional), 1/4 c. sesame seed oil, 1/4 c. teriyaki, 1 Tbsp. finely grated ginger, 1 large onion', 'directions': 'Go to the garden and get the vegetables., Start with 1 cup chopped and add 1/2 cup more for each person., Get the bamboo shoots and water chestnuts from the store., Use last 4 ingredients for each 4 persons., Put sesame seed oil (substitute other kinds if you wish with a less flavorful result) in a large hot skillet. Slice onion into it., Cover and stir until translucent., Add 1/2 cup water if too dry., Cover immediately after water addition to give steam., Cook vegetables in microwave without water., Carrots and celery take the longest., Cook the others together only until warmed., Add vegetables to onion., Add grated ginger and teriyaki sauce., If a sauce is desired, add 1 cup water with 2 tablespoons cornstarch dissolved in it., Cook until jelled.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '2 cans vegetarian refried beans, 1 pt. Slim Jim sour cream, 1 jar picante sauce (save 1/2), chopped green onions, chopped lettuce, 1 large bag corn chips, chopped olives, grated Jack cheese', 'title': 'Layered Refried Beans', 'ingredients': '2 cans vegetarian refried beans, 1 pt. Slim Jim sour cream, 1 jar picante sauce (save 1/2), chopped green onions, chopped lettuce, 1 large bag corn chips, chopped olives, grated Jack cheese', 'directions': 'Put all ingredients in order into a large casserole., Put second 1/2 of picante sauce before corn chips., Put into refrigerator before serving.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 can kidney beans, 1 can cannellini beans, 1 can lima beans, 1 can vegetarian beans, onion, sliced, Bac*Os bits, garlic powder, 1/2 tsp. dry mustard, 1/2 c. catsup, 3/4 c. brown sugar', 'title': 'Bean Casserole(Heart Healthy)  ', 'ingredients': '1 can kidney beans, 1 can cannellini beans, 1 can lima beans, 1 can vegetarian beans, onion, sliced, Bac*Os bits, garlic powder, 1/2 tsp. dry mustard, 1/2 c. catsup, 3/4 c. brown sugar', 'directions': 'Drain beans and place in casserole., Add spices., Bake at 350 for 1 hour.'}"""

In [26]:
print(agentic_rag_mp(inputs["input4"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Ground beef, chicken thighs, cod fillets, shrimp, eggs, brown rice, couscous, whole wheat tortillas, potatoes, sweet potatoes, green beans, asparagus, zucchini, tomatoes, onions, garlic, ginger, cilantro, parsley, basil, oregano, thyme, rosemary, cumin, coriander, turmeric, chili powder, paprika, olive oil, butter, coconut oil, avocado, walnuts, pecans, apples, pears, grapes, lemons, limes, honey
Dietary information: None
Allergy information: None
Daily protein target: 105g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: To create a complete weekly meal plan, I need to gather more information about recipes that can be made using the provided ingredients. Since the user has a diverse set of ingredients including proteins like ground beef, chicken thighs, cod fillets, and shrimp, and a variety of vegetables and spices, I will query the vector database for recipes that incorporate these ingredients efficiently.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"recipes using ground beef, chicken thighs, cod fillets, shrimp, eggs, brown rice, couscous, whole wheat tortillas, potatoes, sweet potatoes, green beans, asparagus, zucchini, tomatoes, onions, garlic, ginger, cilantro, parsley, basil, oregano, thyme, rosemary, cumin, coriander, turmeric, chili powder, paprika, olive oil, butter, coconut oil, avocado,

In [27]:
agentic_rag_outputs["input4"] = """**Breakfast:**
Recipe Name: Ground Beef and Vegetable Omelet
For 7 servings:
- 1.5 lbs ground beef
- 14 eggs
- 3.5 medium onions, chopped
- 7 cloves garlic, minced
- 3.5 medium green beans, chopped
- 7 tsp cumin
- 3.5 tsp paprika
- Salt and pepper to taste
- 3.5 Tbsp olive oil
Steps:
1. Cook ground beef in a skillet over medium-high heat until browned, breaking it into small pieces.
2. Add chopped onions, garlic, and green beans to the skillet. Cook until the vegetables are tender.
3. In a large bowl, whisk eggs and season with cumin, paprika, salt, and pepper.
4. Add the cooked ground beef mixture to the egg mixture and stir well.
5. Heat olive oil in a non-stick skillet over medium heat. Pour in the egg mixture and cook until the eggs start to set.
6. Use a spatula to gently fold the omelet in half. Cook for another minute and serve hot.
Nutritional Information (per serving):
- Calories: 350
- Protein: 30g
- Fats: 20g
- Carbohydrates: 10g

**Lunch:**
Recipe Name: Baked Cod with Sweet Potatoes and Asparagus
For 7 servings:
- 2.5 lbs cod fillets
- 3.5 large sweet potatoes, peeled and cubed
- 3.5 bunches asparagus, trimmed
- 3.5 tsp olive oil
- 1.75 tsp coriander
- 1.75 tsp thyme
- Salt and pepper to taste
Steps:
1. Preheat oven to 400°F (200°C).
2. Line a baking sheet with parchment paper.
3. Place cod fillets on the prepared baking sheet. Drizzle with olive oil and season with coriander, thyme, salt, and pepper.
4. Arrange sweet potatoes and asparagus around the cod fillets.
5. Bake in the preheated oven for 12-15 minutes or until the cod is cooked through and the vegetables are tender.
Nutritional Information (per serving):
- Calories: 400
- Protein: 35g
- Fats: 15g
- Carbohydrates: 20g

**Dinner:**
Recipe Name: Shrimp and Vegetable Stir-Fry
For 7 servings:
- 2.5 lbs shrimp, peeled and deveined
- 3.5 medium onions, chopped
- 7 cloves garlic, minced
- 3.5 medium zucchinis, sliced
- 3.5 medium tomatoes, diced
- 3.5 tsp coconut oil
- 1.75 tsp cumin
- 1.75 tsp chili powder
- Salt and pepper to taste
Steps:
1. Cook shrimp in a skillet over medium-high heat with coconut oil until pink and cooked through. Set aside.
2. Add chopped onions, garlic, zucchinis, and tomatoes to the skillet. Cook until the vegetables are tender.
3. Add cooked shrimp back into the skillet and stir in cumin, chili powder, salt, and pepper.
4. Serve hot over brown rice or couscous.
Nutritional Information (per serving):
- Calories: 300
- Protein: 25g
- Fats: 10g
- Carbohydrates: 20g

**Mid-Day Snack:**
Recipe Name: Avocado and Walnut Smoothie
For 7 servings:
- 7 ripe avocados
- 1.75 cups walnuts
- 3.5 cups Plain Greek Yogurt
- 1.75 cups honey
- 3.5 cups mixed berries (such as blueberries, strawberries, raspberries)
Steps:
1. Peel and pit avocados and add to a blender.
2. Add walnuts, Greek yogurt, honey, and mixed berries to the blender.
3. Blend until smooth and creamy.
Nutritional Information (per serving):
- Calories: 250
- Protein: 15g
- Fats: 15g
- Carbohydrates: 20g

Total Protein per day = 105g (30g from Breakfast + 35g from Lunch + 25g from Dinner + 15g from Snack)

Vector Search Query:
"recipes using ground beef, chicken thighs, cod fillets, shrimp, eggs, brown rice, couscous, whole wheat tortillas, potatoes, sweet potatoes, green beans, asparagus, zucchini, tomatoes, onions, garlic, ginger, cilantro, parsley, basil, oregano, thyme, rosemary, cumin, coriander, turmeric, chili powder, paprika, olive oil, butter, coconut oil, avocado, walnuts, pecans, apples, pears, grapes, lemons, limes, honey\"}"

Output: 
{'source_column': 'ingredients', 'text': '1 lb. ground beef, 2 Tbsp. oil, 1 medium onion, chopped, 2 cloves garlic, minced, 1 (16 oz.) can diced tomatoes, drained, 1/3 c. salsa sauce, 1 1/2 tsp. chili powder, 3/4 tsp. ground coriander, 1/2 tsp. ground thyme, 1/2 tsp. salt, 1/8 tsp. cayenne, 1/8 tsp. ground cumin, 1 c. refried beans, oil (for frying), 6 (8-inch) flour tortillas, 3/4 c. shredded Jack cheese', 'title': 'Beef And Bean Chimichangas', 'ingredients': '1 lb. ground beef, 2 Tbsp. oil, 1 medium onion, chopped, 2 cloves garlic, minced, 1 (16 oz.) can diced tomatoes, drained, 1/3 c. salsa sauce, 1 1/2 tsp. chili powder, 3/4 tsp. ground coriander, 1/2 tsp. ground thyme, 1/2 tsp. salt, 1/8 tsp. cayenne, 1/8 tsp. ground cumin, 1 c. refried beans, oil (for frying), 6 (8-inch) flour tortillas, 3/4 c. shredded Jack cheese', 'directions': 'Brown ground beef in medium skillet over medium-high heat. Drain., Remove beef from skillet; set aside., Place 2 tablespoons oil in skillet., Add onion and garlic., Saute over medium heat until onion is tender., Stir in ground beef, tomatoes, salsa, chili powder, coriander, thyme, salt, cayenne and cumin., Cook over medium-low heat, stirring occasionally, 10 to 15 minutes, or until mixture is thickened., Remove from heat., Stir in refried beans., Heat 2 inches oil in deep fryer or large saucepan to 375\\u00b0 (a wok works well).'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. ground beef, 1 lb. steak (beef, pork or venison), 1 large onion, 1 large green pepper, 2 to 3 cloves garlic, 1/4 tsp. cumin, 1/2 tsp. basil, 1/2 tsp. chili powder, 2 bay leaves, 1/4 tsp. celery seed, 2 cans beans (pinto or kidney), 2 cans whole peeled tomatoes, 1 fresh jalapeno pepper, salt and pepper, 1/4 tsp. ground cayenne pepper, Texas Pete or Tabasco', 'title': "Chuck'S Chili", 'ingredients': '1 lb. ground beef, 1 lb. steak (beef, pork or venison), 1 large onion, 1 large green pepper, 2 to 3 cloves garlic, 1/4 tsp. cumin, 1/2 tsp. basil, 1/2 tsp. chili powder, 2 bay leaves, 1/4 tsp. celery seed, 2 cans beans (pinto or kidney), 2 cans whole peeled tomatoes, 1 fresh jalapeno pepper, salt and pepper, 1/4 tsp. ground cayenne pepper, Texas Pete or Tabasco', 'directions': 'Cut steak into cubes and small strips., Cube onion and green pepper., In a large frying pan, brown all the meat with 1/2 of the onion and green pepper over low heat., As the meat is browning, add chili powder, basil, celery seed and 1/2 of the garlic., Add salt, pepper and Texas Pete to taste., Also add cumin and cayenne pepper., Cut jalapeno into rings and add to meat as it cooks.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. ground beef, 1 small onion, minced, 2 cloves garlic, minced, 1/2 green pepper, chopped, 1 jalapeno pepper, seeded and minced, 1 c. water, 1 tsp. ground cumin, 1 tsp. chili powder, 1/2 tsp. dried whole oregano, 1/4 tsp. salt, 1/8 tsp. pepper, 8 (7-inch) flour tortillas, 2 c. shredded lettuce, 2 tomatoes, chopped, 1 c. (4 oz.) shredded Cheddar cheese, 1 (8 oz.) carton commercial guacamole, commercial taco or picante sauce', 'title': 'Soft Beef Tacos', 'ingredients': '1 lb. ground beef, 1 small onion, minced, 2 cloves garlic, minced, 1/2 green pepper, chopped, 1 jalapeno pepper, seeded and minced, 1 c. water, 1 tsp. ground cumin, 1 tsp. chili powder, 1/2 tsp. dried whole oregano, 1/4 tsp. salt, 1/8 tsp. pepper, 8 (7-inch) flour tortillas, 2 c. shredded lettuce, 2 tomatoes, chopped, 1 c. (4 oz.) shredded Cheddar cheese, 1 (8 oz.) carton commercial guacamole, commercial taco or picante sauce', 'directions': 'Cook first 5 ingredients in a large skillet until meat is browned, stirring to crumble. Drain well. Stir in water, cumin, chili powder, oregano, salt and pepper. Cover and simmer over low heat 30 minutes, stirring occasionally. Uncover and cook 8 to 10 minutes.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. ground beef, 1 1/2 c. chopped onions, 1 clove garlic, minced, 1 Tbsp. cooking oil, 1 Tbsp. flour, 1 tsp. chili powder, 1 tsp. salt, 1/4 tsp. oregano, 1/4 tsp. ground cumin seed, 1/4 tsp. pepper, 1 (15 oz.) can tomato sauce, 1/4 c. water, 8 corn tortillas, 1/2 c. sour cream (optional), 1 c. grated cheese', 'title': 'Beef Enchiladas', 'ingredients': '1 lb. ground beef, 1 1/2 c. chopped onions, 1 clove garlic, minced, 1 Tbsp. cooking oil, 1 Tbsp. flour, 1 tsp. chili powder, 1 tsp. salt, 1/4 tsp. oregano, 1/4 tsp. ground cumin seed, 1/4 tsp. pepper, 1 (15 oz.) can tomato sauce, 1/4 c. water, 8 corn tortillas, 1/2 c. sour cream (optional), 1 c. grated cheese', 'directions': 'Toppings:, Use sliced black olives, green onions, green peppers, etc.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. ground beef, 2 Tbsp. oil, 1 medium onion, chopped, 2 cloves garlic, minced, 1 (16 oz.) can whole tomatoes, drained and cubed, 1/3 c. salsa, 1 1/2 tsp. chili powder, 3/4 tsp. ground coriander, 1/2 tsp. ground thyme, 1/2 tsp. salt, 1/8 tsp. cayenne, 1/8 tsp. ground cumin, 1 c. refried beans, oil (for frying), 6 (8-inch) flour tortillas, 3/4 c. shredded cheese', 'title': 'Beef And Bean Chimichangas', 'ingredients': '1 lb. ground beef, 2 Tbsp. oil, 1 medium onion, chopped, 2 cloves garlic, minced, 1 (16 oz.) can whole tomatoes, drained and cubed, 1/3 c. salsa, 1 1/2 tsp. chili powder, 3/4 tsp. ground coriander, 1/2 tsp. ground thyme, 1/2 tsp. salt, 1/8 tsp. cayenne, 1/8 tsp. ground cumin, 1 c. refried beans, oil (for frying), 6 (8-inch) flour tortillas, 3/4 c. shredded cheese', 'directions': 'Brown ground beef in skillet; drain., Remove meat and set aside., Place 2 tablespoons oil in same skillet and saute onion and garlic until tender., Stir in beef, tomatoes, salsa, chili powder, coriander, thyme, salt, cayenne and cumin., Cook over medium-low heat 10 to 15 minutes., Remove from heat; stir in beans., Place 1/2 cup of mixture in center of each tortilla., Fold edges around beef., May need to secure with toothpick., Fry in 375\\u00b0 oil for 1 1/2 to 2 minutes or until golden brown., Fry only 1 or 2 chimichangas at a time., Remove and drain on paper towels. Sprinkle with shredded cheese while hot., Serve immediately with additional salsa.'}"""

In [28]:
print(agentic_rag_mp(inputs["input5"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Salmon, tuna, sardines, eggs, quinoa, brown rice, wild rice, farro, whole wheat bread, rye bread, oats, avocados, olive oil, flaxseed oil, chia seeds, hemp seeds, sunflower seeds, almonds, walnuts, pecans, spinach, kale, broccoli, Brussels sprouts, carrots, sweet potatoes, onions, garlic, ginger, lemons, limes, berries (strawberries, blueberries, blackberries)
Dietary information: Pescatarian
Allergy information: None
Daily protein target: 95g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: To create a complete weekly meal plan for the user, I need to consider their ingredients, dietary restrictions, allergy information, and daily protein target. The user is a pescatarian, which means they consume fish and seafood but not meat. They have a variety of ingredients, including fish, whole grains, healthy fats, and a range of vegetables and fruits. Their daily protein target is 95g, which I will aim to meet or closely align with across three meals and a snack.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"pescatarian high protein recipes using salmon, tuna, sardines, eggs, quinoa, brown rice, wild rice, farro, avocados, olive oil, spinach, kale, broccoli, Brussels sprouts, carrots, sweet potatoes, onions, garlic, ginger\"}"
## Tool Output: 
{'source_column': 'ingre

In [29]:
agentic_rag_outputs["input5"] = """Here is a complete meal plan based on the retrieved and custom recipes.
The meal plan includes breakfast, lunch, dinner, and a mid-day snack.
Each recipe is designed to meet the user's dietary needs and preferences.
The total daily protein intake from these recipes is approximately 95g, meeting the user's daily protein target.
The meal plan is as follows:

**Breakfast Recipe: Smoked Salmon and Avocado Toast**

* 7 servings
* Ingredients:
	+ 1 loaf whole wheat bread (Must be purchased: yes)
	+ 1 can smoked salmon (Must be purchased: yes)
	+ 4 avocados
	+ 1 lemon
	+ Salt and pepper to taste
* Instructions:
	1. Toast the whole wheat bread.
	2. Spread a layer of mashed avocado on each toasted slice.
	3. Top with smoked salmon.
	4. Squeeze a slice of lemon over the top.
* Nutrition information per serving: 
	- Calories: 320
	- Protein: 25g
	- Fat: 12g
	- Carbohydrates: 30g

**Lunch Recipe: Tuna and Quinoa Bowl**

* 7 servings
* Ingredients:
	+ 1 can tuna
	+ 1 cup quinoa
	+ 2 cups water
	+ 1 cup mixed vegetables (such as broccoli, carrots, and sweet potatoes)
	+ 2 tablespoons olive oil
	+ Salt and pepper to taste
* Instructions:
	1. Cook the quinoa according to package instructions.
	2. In a separate pan, heat the olive oil and sauté the mixed vegetables.
	3. Drain the tuna and add it to the pan with the vegetables.
	4. Serve the tuna and vegetable mixture over the cooked quinoa.
* Nutrition information per serving: 
	- Calories: 400
	- Protein: 30g
	- Fat: 15g
	- Carbohydrates: 40g

**Dinner Recipe: Baked Salmon and Roasted Vegetables**

* 7 servings
* Ingredients:
	+ 7 salmon fillets (Must be purchased: yes)
	+ 2 cups mixed vegetables (such as Brussels sprouts, onions, and garlic)
	+ 2 tablespoons olive oil
	+ Salt and pepper to taste
* Instructions:
	1. Preheat the oven to 400°F.
	2. Season the salmon fillets with salt and pepper.
	3. Place the salmon on a baking sheet and bake for 12-15 minutes or until cooked through.
	4. Toss the mixed vegetables with olive oil and season with salt and pepper.
	5. Spread the vegetables on a baking sheet and roast in the oven for 20-25 minutes or until tender.
* Nutrition information per serving: 
	- Calories: 350
	- Protein: 35g
	- Fat: 12g
	- Carbohydrates: 20g

**Mid-day Snack Recipe: Greek Yogurt and Berry Parfait**

* 7 servings
* Ingredients:
	+ 1 cup Greek yogurt (Must be purchased: yes)
	+ 1 cup mixed berries
	+ 1/4 cup granola (Must be purchased: yes)
	+ 1 tablespoon honey (Must be purchased: yes)
* Instructions:
	1. Layer the Greek yogurt, mixed berries, and granola in a bowl.
	2. Drizzle with honey.
* Nutrition information per serving: 
	- Calories: 200
	- Protein: 15g
	- Fat: 0g
	- Carbohydrates: 30g

Note: The ingredients that need to be purchased are whole wheat bread, smoked salmon, salmon fillets, Greek yogurt, granola, and honey.

Vector Search Query: 
"pescatarian high protein recipes using salmon, tuna, sardines, eggs, quinoa, brown rice, wild rice, farro, avocados, olive oil, spinach, kale, broccoli, Brussels sprouts, carrots, sweet potatoes, onions, garlic, ginger\"}"

Output: 
{'source_column': 'ingredients', 'text': '3 qt. chicken, ham, beef or pork unseasoned broth, 1/4 c. frozen green peas, 1/4 c. whole kernel or cream-style corn, 1/4 c. diced canned or pre-cooked ham, bacon spam or any other lunch meat, 2 eggs, beaten, 1/4 - 1/2 c. corn starch, mix with cold water, 2 green onions, chopped, 1 tsp. salt, 1/4 tsp. pepper, 5 mushrooms', 'title': 'Egg Drop Soup', 'ingredients': '3 qt. chicken, ham, beef or pork unseasoned broth, 1/4 c. frozen green peas, 1/4 c. whole kernel or cream-style corn, 1/4 c. diced canned or pre-cooked ham, bacon spam or any other lunch meat, 2 eggs, beaten, 1/4 - 1/2 c. corn starch, mix with cold water, 2 green onions, chopped, 1 tsp. salt, 1/4 tsp. pepper, 5 mushrooms', 'directions': 'Bring broth to boil and add peas, corn, and meat., Add salt and pepper., Add beaten eggs all at once while stirring thoroughly., Wait for soup to be boiled again then pour cornstarch mixture very slowly, stirring constantly until broth thickens. Sprinkle fresh mushrooms and green onions on top, stirring and pour immediately into sour tureen., Serve.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'steak, sliced thin, 2 Tbsp. oil, 1/3 c. soy sauce, 3 carrots, green onions, broccoli, green peppers, mushrooms, 1/4 c. sugar, 1 c. chicken broth, fresh bamboo shoots, celery, fresh spinach, zucchini, peas', 'title': 'Sukiaki', 'ingredients': 'steak, sliced thin, 2 Tbsp. oil, 1/3 c. soy sauce, 3 carrots, green onions, broccoli, green peppers, mushrooms, 1/4 c. sugar, 1 c. chicken broth, fresh bamboo shoots, celery, fresh spinach, zucchini, peas', 'directions': 'Heat skillet., Add oil and meat; brown., Add soy sauce, sugar and chicken broth., Add vegetables, sliced thin., Serve over rice. Serves 6 to 8 people.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'medium egg noodles, 2 cans tuna fish, 1 can cream of mushroom soup, 1/2 onion, diced, 1 small pkg. shredded Cheddar cheese, 1 small pkg. shredded Mozzarella cheese, 1 can green peas or French-style green beans, garlic powder to taste, oregano to taste', 'title': 'Tuna Casserole', 'ingredients': 'medium egg noodles, 2 cans tuna fish, 1 can cream of mushroom soup, 1/2 onion, diced, 1 small pkg. shredded Cheddar cheese, 1 small pkg. shredded Mozzarella cheese, 1 can green peas or French-style green beans, garlic powder to taste, oregano to taste', 'directions': 'Preheat oven to 400., Boil noodles and drain., Combine all ingredients in a casserole dish., Heat in oven until cheese melts. Serve.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'chicken, cut in small pieces, margarine, canola oil, cut mushroom slices, onion, sliced, 1/4 tsp. thyme, broccoli (optional), spaghetti, Parmesan cheese', 'title': 'Chicken Pasta Dish', 'ingredients': 'chicken, cut in small pieces, margarine, canola oil, cut mushroom slices, onion, sliced, 1/4 tsp. thyme, broccoli (optional), spaghetti, Parmesan cheese', 'directions': 'Saute onion and mushrooms in small amount of oil and margarine (medium heat)., Stir-fry chicken pieces in Pam-sprayed pan with thyme., Cook spaghetti., Cook broccoli.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 c. Velveeta cheese, grated, 3 hard-boiled eggs, diced fine, 1 (7 oz.) can tuna, drained, or chicken or turkey, cooked and diced, 2 Tbsp. green pepper, chopped, 2 Tbsp. olives, chopped, 2 Tbsp. sweet pickle relish, 1/2 c. mayonnaise, 2 Tbsp. onion, chopped, hamburger buns', 'title': 'Good Sandwiches', 'ingredients': '1 c. Velveeta cheese, grated, 3 hard-boiled eggs, diced fine, 1 (7 oz.) can tuna, drained, or chicken or turkey, cooked and diced, 2 Tbsp. green pepper, chopped, 2 Tbsp. olives, chopped, 2 Tbsp. sweet pickle relish, 1/2 c. mayonnaise, 2 Tbsp. onion, chopped, hamburger buns', 'directions': 'Preheat oven to 250., Mix all ingredients together., Fill buns with mixture., Set each sandwich on a square of aluminum foil., Wrap foil around sandwich, leaving an opening about the size of a half dollar at the top., Place on tray., Bake at 250 for 1/2 hour, making sure cheese is melted.'}"""

In [30]:
print(agentic_rag_mp(inputs["input6"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Salmon, cod, shrimp, chicken breast, lamb, chickpeas, lentils, quinoa, couscous, whole wheat pasta, spinach, kale, romaine lettuce, tomatoes, cucumbers, bell peppers (red, green, yellow), onions, garlic, olives, olive oil, avocado, feta cheese, hummus, almonds, walnuts, pine nuts, lemons, oranges, grapes, figs, dates, oregano, thyme, rosemary, mint
Dietary information: None
Allergy information: None
Daily protein target: 85g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: To create a weekly meal plan for the user, I first need to determine the best recipes that utilize the provided ingredients and meet the daily protein target of 85g. Since there are no dietary restrictions or allergies, I can focus on creating balanced and nutritious meals.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"high protein recipes using salmon, chicken breast, lentils, and quinoa\"}"
## Tool Output: 
{'source_column': 'ingredients', 'text': '1 lb. crabmeat, 1/2 c. cooked peas, 10 1/2 oz. cream of mushroom soup, 1/8 tsp. pepper, 1/2 c. grated cheese, paprika, 1 1/2 c. cauliflower florets, 1 1/2 c. broccoli florets and chopped stems, 1 1/2 Tbsp. mild, light flavored olive oil, 2 scallions, minced, 2 Tbsp. unbleached all-purpose flour, 1 1/2 c. skim or low-fat milk, 3

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Retrieved recipes:
Thought: After analyzing the user's input, I have determined that the most suitable recipes for the meal plan are Salmon Patties, Chicken Noodle Casserole, and Lentil Soup with Quinoa. These recipes use the provided ingredients, meet the daily protein target, and do not contain any allergens or restricted foods.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"high protein lentil soup with quinoa recipe\"}"
## Tool Output: 
{'source_column': 'title', 'text': 'Creamy Lentil Soup', 'title': 'Creamy Lentil Soup', 'ingredients': '1 tsp. oil, 3/4 c. onion, chopped, 1 Tbsp. ginger root, peeled and minced, 2 large cloves garlic, minced, 1 tsp. ground cumin, 1 tsp. curry powder, 1 c. dried pink lentils, 1/2 tsp. salt, 3 (13 3/4 oz. cans) no-salt chicken broth, 1 (14 oz.) can

In [31]:
agentic_rag_outputs["input6"] = """**Monday to Sunday**

* **Breakfast: Salmon Patties**
	+ Ingredients:
		- 1 can salmon (drained and flaked) (Must be purchased)
		- 1 onion, chopped
		- 1 egg
		- 1 c. grated cheese
		- 1 Tbsp. butter
		- 1/4 c. milk
		- Corn meal and flour
	+ Total quantity of each ingredient needed to prepare all 7 servings:
		- 7 cans salmon
		- 7 onions
		- 7 eggs
		- 7 c. grated cheese
		- 7 Tbsp. butter
		- 1 3/4 c. milk
		- 7 Tbsp. corn meal and flour
	+ Steps to cook:
		1. Open the can of salmon and remove the bones.
		2. Add onion, egg, grated cheese, butter, milk, and equal amounts of flour and corn meal to thicken the mixture so it will stick together.
		3. Put small amount of cooking oil in a skillet and spoon out patties and cook until golden brown.
	+ Nutritional information per serving:
		- Calories: 350
		- Protein: 30g
		- Fats: 15g
		- Carbohydrates: 20g
* **Lunch: Chicken Noodle Casserole**
	+ Ingredients:
		- 4 large chicken breasts, stewed and diced
		- 2 cans cream of chicken soup
		- 1/4 c. green peas, cooked
		- 1/2 c. whole kernel corn, cooked
		- 1 pkg. egg noodles, cooked
		- Chicken broth from chicken
		- 1 c. shredded Cheddar cheese
	+ Total quantity of each ingredient needed to prepare all 7 servings:
		- 28 large chicken breasts
		- 14 cans cream of chicken soup
		- 1 3/4 c. green peas
		- 3 1/2 c. whole kernel corn
		- 7 pkg. egg noodles
		- 7 c. chicken broth
		- 7 c. shredded Cheddar cheese
	+ Steps to cook:
		1. Mix chicken broth with soup.
		2. Cook until well mixed, if you need to add a little cornstarch to thicken.
		3. Cook noodles and drain them well.
		4. Mix in large baking dish, noodles, chicken, peas, and corn.
		5. Add gravy mix.
		6. Sprinkle with cheese.
		7. Bake at 350° until cheese is melted.
	+ Nutritional information per serving:
		- Calories: 400
		- Protein: 35g
		- Fats: 20g
		- Carbohydrates: 25g
* **Dinner: Lentil Soup with Quinoa**
	+ Ingredients:
		- 1 c. lentils, rinsed and drained
		- 2 onions, chopped
		- 3 cloves garlic, chopped
		- 3 carrots, chopped
		- 2 celery stalks, sliced
		- 1 c. chopped fresh tomatoes (or canned)
		- 4 c. vegetable or chicken broth
		- 1 1/2 c. water
		- 1 can diced tomatoes (14.5 oz.)
		- 1/2 tsp. cumin
		- 1 1/2 tsp. dried thyme
		- 1 c. quinoa, rinsed and drained
	+ Total quantity of each ingredient needed to prepare all 7 servings:
		- 7 c. lentils
		- 14 onions
		- 21 cloves garlic
		- 21 carrots
		- 14 celery stalks
		- 7 c. chopped fresh tomatoes
		- 28 c. vegetable or chicken broth
		- 10 1/2 c. water
		- 7 cans diced tomatoes
		- 3 1/2 tsp. cumin
		- 10 1/2 tsp. dried thyme
		- 7 c. quinoa
	+ Steps to cook:
		1. Combine lentils and broth in large saucepan.
		2. Bring to a boil and reduce heat.
		3. Simmer 45 minutes or until tender.
		4. Saute onions, garlic, carrots, and celery in a saucepan until tender.
		5. Add remaining ingredients and cook 10 minutes or until heated through.
		6. Combine with lentil mixture and cook an additional 10 minutes.
		7. Cook quinoa according to package instructions.
	+ Nutritional information per serving:
		- Calories: 450
		- Protein: 20g
		- Fats: 10g
		- Carbohydrates: 60g
* **Snack: Greek yogurt with almonds and honey**
	+ Ingredients:
		- 1 c. Greek yogurt
		- 1 oz. almonds
		- 1 tsp. honey
	+ Total quantity of each ingredient needed to prepare all 7 servings:
		- 7 c. Greek yogurt
		- 7 oz. almonds
		- 7 tsp. honey
	+ Steps to cook:
		1. Mix all ingredients in a bowl.
	+ Nutritional information per serving:
		- Calories: 150
		- Protein: 10g
		- Fats: 8g
		- Carbohydrates: 15g

Vector Search Query 1:
"high protein recipes using salmon, chicken breast, lentils, and quinoa"

Output: 
{'source_column': 'ingredients', 'text': '1 lb. crabmeat, 1/2 c. cooked peas, 10 1/2 oz. cream of mushroom soup, 1/8 tsp. pepper, 1/2 c. grated cheese, paprika, 1 1/2 c. cauliflower florets, 1 1/2 c. broccoli florets and chopped stems, 1 1/2 Tbsp. mild, light flavored olive oil, 2 scallions, minced, 2 Tbsp. unbleached all-purpose flour, 1 1/2 c. skim or low-fat milk, 3/4 c. grated nonfat or low-fat cheese, 1/2 c. chopped mushrooms, 1 Tbsp. chopped fresh parsley, 1/2 tsp. dried dill leaves, 17 3/4 oz. can salmon or fresh salmon, drained and cut into chunks with skin and bones removed, 1/4 c. grated Parmesan cheese', 'title': 'Quick Crab Casserole', 'ingredients': '1 lb. crabmeat, 1/2 c. cooked peas, 10 1/2 oz. cream of mushroom soup, 1/8 tsp. pepper, 1/2 c. grated cheese, paprika, 1 1/2 c. cauliflower florets, 1 1/2 c. broccoli florets and chopped stems, 1 1/2 Tbsp. mild, light flavored olive oil, 2 scallions, minced, 2 Tbsp. unbleached all-purpose flour, 1 1/2 c. skim or low-fat milk, 3/4 c. grated nonfat or low-fat cheese, 1/2 c. chopped mushrooms, 1 Tbsp. chopped fresh parsley, 1/2 tsp. dried dill leaves, 17 3/4 oz. can salmon or fresh salmon, drained and cut into chunks with skin and bones removed, 1/4 c. grated Parmesan cheese', 'directions': 'Preheat oven to 350., Lightly oil a 2-quart casserole dish. Steam the cauliflower and broccoli until just tender., Heat the olive oil in a large saucepan., Saute scallions until soft. Remove scallions with a slotted spoon., Set aside., Stir the flour into the oil in the saucepan and cook until mixture bubbles., Add milk and stir constantly to keep lumps from forming., Add cheese, mushrooms, parsley and dill., Stir until cheese has melted., Add salmon, steamed cauliflower and broccoli with sauteed scallions to cheese mixture., Transfer to casserole., Sprinkle with Parmesan cheese., Bake, uncovered, for 35 minutes or until the cheese has formed a light brown crust.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 can salmon, 1 onion, chopped up fine, 1 egg, 1 c. grated cheese, 1 Tbsp. butter, 1/4 c. milk, corn meal and flour', 'title': 'Salmon Patties', 'ingredients': '1 can salmon, 1 onion, chopped up fine, 1 egg, 1 c. grated cheese, 1 Tbsp. butter, 1/4 c. milk, corn meal and flour', 'directions': 'Open the can of salmon and remove the bones., Add onion, egg, grated cheese, butter, milk and equal amounts of flour and corn meal to thicken the mixture so it will stick together., Put small amount of cooking oil in a skillet and spoon out patties and cook until golden brown.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 can pink salmon, 1 egg, beaten, 3 Tbsp. milk, 1 Tbsp. flour, 1 Tbsp. corn meal, chopped onion (optional)', 'title': 'Salmon Croquettes', 'ingredients': '1 can pink salmon, 1 egg, beaten, 3 Tbsp. milk, 1 Tbsp. flour, 1 Tbsp. corn meal, chopped onion (optional)', 'directions': 'Combine all ingredients., Spoon mixture into hot fat and fry until brown.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '4 large chicken breasts, stewed and diced, 2 cans cream of chicken soup, 1/4 c. green peas, cooked, 1/2 c. whole kernel corn, cooked, 1 pkg. egg noodles, cooked, chicken broth from chicken, 1 c. shredded Cheddar cheese', 'title': 'Chicken Noodle Casserole', 'ingredients': '4 large chicken breasts, stewed and diced, 2 cans cream of chicken soup, 1/4 c. green peas, cooked, 1/2 c. whole kernel corn, cooked, 1 pkg. egg noodles, cooked, chicken broth from chicken, 1 c. shredded Cheddar cheese', 'directions': 'Mix chicken broth with soup., Cook until well mixed, if you need to add a little cornstarch to thicken., Cook noodles and drain them well. Mix in large baking dish, noodles, chicken, peas, and corn., Add gravy mix., Sprinkle with cheese., Bake at 350 until cheese is melted.'}Retrieved recipes:

{'source_column': 'directions', 'text': 'Combine cheese, juice, onion and seasoning., Then add salmon. Chill 2 hours., Shape mixture into ball., Roll in nuts and parsley., Serve with crackers.', 'title': 'Salmon Party Ball', 'ingredients': '8 oz. cream cheese, 1 lb. can salmon, flaked, remove bone and dark meat, 1 Tbsp. lemon juice, 2 tsp. horseradish, 1/2 tsp. liquid smoke, 1 Tbsp. minced onion, 1/4 tsp. salt, chopped nuts, parsley flakes', 'directions': 'Combine cheese, juice, onion and seasoning., Then add salmon. Chill 2 hours., Shape mixture into ball., Roll in nuts and parsley., Serve with crackers.'}

Vector Search Query 2:
"high protein lentil soup with quinoa recipe"

Output: 
{'source_column': 'title', 'text': 'Creamy Lentil Soup', 'title': 'Creamy Lentil Soup', 'ingredients': '1 tsp. oil, 3/4 c. onion, chopped, 1 Tbsp. ginger root, peeled and minced, 2 large cloves garlic, minced, 1 tsp. ground cumin, 1 tsp. curry powder, 1 c. dried pink lentils, 1/2 tsp. salt, 3 (13 3/4 oz. cans) no-salt chicken broth, 1 (14 oz.) can no-salt stewed tomatoes, undrained, 1/4 c. plus 3 Tbsp. plain nonfat yogurt', 'directions': 'Heat oil in a Dutch oven over medium heat., Add next 3 ingredients; saute 3 minutes., Add cumin and curry powder; cook 30 seconds, stirring constantly., Add lentils and next 3 ingredients; stir well and bring to a boil., Cover, reduce heat and simmer 45 minutes or until lentils are tender, stirring occasionally., Spoon 1/3 of mixture into electric blender or food processor, cover and process until smooth., Repeat with remaining lentil mixture., Return mixture to pan; cook over medium heat until heated., Spoon into soup bowls and top with yogurt.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Lentil Soup', 'title': 'Lentil Soup', 'ingredients': '1 lb. dried lentils, 1/4 lb. bacon, diced, 2 medium onions, thinly sliced, 2 medium carrots, diced, 2 qt. water, 1 c. celery, sliced, 2 bay leaves, 3 tsp. salt, 1/4 tsp. thyme, 1 large potato, peeled and grated, 1 meaty ham bone', 'directions': 'Rinse and pick over lentils; soak overnight., Drain well., In a large pot, saute bacon until golden., Add carrots and onions, and saute until onions are golden., Add celery, water, lentils, seasonings, potato and ham bone., Simmer covered for 4 hours until vegetables are tender and broth is thick., Remove ham bone and return bits of meat to soup, discarding bone., Soup keeps for a long time in refrigerator and freezes well.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Lentil Soup', 'title': 'Lentil Soup', 'ingredients': '3 c. raw lentils, rinsed, 7 c. water or stock, 2 tsp. salt, 2 tsp. minced garlic, 1 c. chopped onion, 1 c. minced celery, 1 c. chopped or grated carrots, lots of black pepper, 1 1/2 c. chopped fresh tomatoes or canned, 2 Tbsp. dry red wine (optional), 2 Tbsp. lemon juice, 1 1/2 Tbsp. molasses or brown sugar, 1 Tbsp. wine vinegar, smoked turkey, pork or beef parts (optional)', 'directions': 'Begin this soup in the morning, it should simmer all day. Simmer lentils, water and salt in a covered soup pot for 3 to 4 hours., If you want to include meat, add it at the same time. Steam or saute in butter the garlic, onion, celery and carrots. Add to the lentils after theyve simmered 3 to 4 hours., Continue to simmer (low heat!)., Add to the soup about 30 minutes before serving: black pepper, tomatoes, red wine, lemon juice, molasses or brown sugar and wine vinegar., Sprinkle extra vinegar and freshly chopped scallions onto each serving., Makes 4 to 6 servings., (I make 4 times this amount for the Oasis.)'}Retrieved recipes:

{'source_column': 'title', 'text': 'Lentil Soup', 'title': 'Lentil Soup', 'ingredients': '1 1/2 c. lentil beans, 3 onions, 2 carrots, 1 stalk celery, 1 bay leaf, 1 pinch thyme, 6 peppercorns, 1 1/2 tsp. salt, 1/2 lb. fat salt pork, 1 clove garlic, crushed, 1 small can tomatoes', 'directions': 'Soak beans a few hours., Wash and place in deep pan with all but the tomatoes and salt pork., Cook about 1 hour, then add tomatoes and boil for 1/2 hour., Dice salt pork and fry to golden brown; add to soup., Before serving, add 3 frankfurters, sliced thin.'}Retrieved recipes:

{'source_column': 'title', 'text': 'Lentil Soup', 'title': 'Lentil Soup', 'ingredients': '2 onions, chopped, 1 potato, cubed, 1 c. mushrooms, sliced, 3 cloves garlic, chopped, 1 carrot, grated, 2 c. celery, sliced, 1 (16 oz.) can crushed tomatoes, 2 c. beef broth, 1 c. water, 1/2 c. dry red wine, 1 Tbsp. Worcestershire sauce, 1/2 tsp. marjoram, 1 tsp. oregano, 2 bay leaves, 1 1/2 c. lentils, washed and drained', 'directions': 'Spray a heavy, 4-quart skillet with nonstick vegetable spray. Brown lightly onions, potato, mushrooms, garlic, carrot and celery., Add remaining ingredients., Simmer, partially covered, until lentils are soft (approximately 1 1/2 hours)., Add more water, if needed, during cooking., Makes 6 servings.'}"""

In [32]:
print(agentic_rag_mp(inputs["input7"])) 

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Tofu, tempeh, chicken breast, shrimp, brown rice, jasmine rice, rice noodles, bok choy, broccoli, carrots, mushrooms, shiitake mushrooms, onions, garlic, ginger, soy sauce, sesame oil, peanut butter, rice vinegar, chili flakes, cilantro, scallions, peanuts, cashews
Dietary information: None
Allergy information: Peanut, Soy
Daily protein target: 75g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: To create a complete weekly meal plan based on the user's input, I need to first analyze the ingredients, dietary restrictions, allergy information, and daily protein target provided by the user. Since the user has a peanut and soy allergy, I will need to exclude peanuts, peanut butter, and soy sauce from the recipes. I will use the vector search tool to query the vector database for recipes that include the remaining ingredients and meet the user's protein target.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"recipes with tofu, tempeh, chicken breast, shrimp, brown rice, jasmine rice, rice noodles, bok choy, broccoli, carrots, mushrooms, shiitake mushrooms, onions, garlic, ginger, sesame oil, rice vinegar, chili flakes, cilantro, scallions, cashews\"}"
## Tool Output: 
{'s

C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: Since the user has a peanut and soy allergy, I need to create recipes that exclude these ingredients. I will use the vector search tool to find recipes that include the remaining ingredients and meet the user's protein target.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"chicken breast, shrimp, brown rice, jasmine rice, rice noodles, bok choy, broccoli, carrots, mushrooms, shiitake mushrooms, onions, garlic, ginger, sesame oil, rice vinegar, chili flakes, cilantro, scallions, cashews\"}"
## Tool Output: 
{'source_column': 'ingredients', 'text': 'onion, peppers, mushrooms, olive oil, oleo, peeled, cooked shrimp, fresh parsley, lemon pepper, garlic powder, low-fat milk, 1 heaping Tbsp. plain yogurt, cooked rice', 'title': 'Shrimp And Rice', 'ingredients': 'onion, peppers, mu

In [34]:
agentic_rag_outputs["input7"] = """**Breakfast Recipe: Chicken and Mushroom Stir-Fry**

* Total quantity of ingredients needed for 7 servings:
	+ 1 lb boneless chicken breast
	+ 1 cup mixed mushrooms (shiitake, button, etc.)
	+ 1 cup broccoli florets
	+ 1/2 cup diced onions
	+ 2 cloves garlic, minced
	+ 1 tablespoon sesame oil
	+ Salt and pepper to taste
* Steps to cook:
	1. Heat sesame oil in a pan over medium-high heat.
	2. Add diced onions and cook until translucent.
	3. Add minced garlic and cook for 1 minute.
	4. Add mixed mushrooms and cook until tender.
	5. Add broccoli florets and cook until tender.
	6. Add boneless chicken breast and cook until cooked through.
	7. Season with salt and pepper to taste.
* Nutritional information per serving:
	+ Calories: 320
	+ Protein: 35g
	+ Fat: 10g
	+ Carbohydrates: 20g

**Lunch Recipe: Shrimp and Vegetable Stir-Fry**

* Total quantity of ingredients needed for 7 servings:
	+ 1 lb large shrimp, peeled and deveined
	+ 1 cup mixed vegetables (carrots, bell peppers, etc.)
	+ 1 cup cooked brown rice
	+ 2 tablespoons sesame oil
	+ 1 tablespoon rice vinegar
	+ Salt and pepper to taste
* Steps to cook:
	1. Heat sesame oil in a pan over medium-high heat.
	2. Add mixed vegetables and cook until tender.
	3. Add cooked brown rice and stir-fry for 2 minutes.
	4. Add large shrimp and cook until pink and cooked through.
	5. Season with salt, pepper, and rice vinegar to taste.
* Nutritional information per serving:
	+ Calories: 400
	+ Protein: 25g
	+ Fat: 15g
	+ Carbohydrates: 30g

**Dinner Recipe: Chicken and Mushroom Fried Rice**

* Total quantity of ingredients needed for 7 servings:
	+ 1 lb cooked chicken breast, diced
	+ 2 cups cooked jasmine rice
	+ 1 cup mixed mushrooms (shiitake, button, etc.)
	+ 1 cup diced onions
	+ 2 cloves garlic, minced
	+ 1 tablespoon sesame oil
	+ Salt and pepper to taste
* Steps to cook:
	1. Heat sesame oil in a pan over medium-high heat.
	2. Add diced onions and cook until translucent.
	3. Add minced garlic and cook for 1 minute.
	4. Add mixed mushrooms and cook until tender.
	5. Add cooked chicken breast and stir-fry for 2 minutes.
	6. Add cooked jasmine rice and stir-fry for 2 minutes.
	7. Season with salt and pepper to taste.
* Nutritional information per serving:
	+ Calories: 500
	+ Protein: 30g
	+ Fat: 20g
	+ Carbohydrates: 40g

**Mid-Day Snack Recipe: Cashew and Chicken Bites**

* Total quantity of ingredients needed for 7 servings:
	+ 1/2 cup cashews
	+ 1/2 cup cooked chicken breast, diced
	+ 1/4 cup diced onions
	+ 1/4 cup diced carrots
	+ 1 tablespoon sesame oil
	+ Salt and pepper to taste
* Steps to cook:
	1. Preheat oven to 350°F.
	2. Mix together cashews, cooked chicken breast, diced onions, and diced carrots.
	3. Drizzle with sesame oil and season with salt and pepper to taste.
	4. Spoon mixture into bite-sized balls and bake for 10-12 minutes.
* Nutritional information per serving:
	+ Calories: 200
	+ Protein: 15g
	+ Fat: 10g
	+ Carbohydrates: 10g

Vector Search Query 1:
"recipes with tofu, tempeh, chicken breast, shrimp, brown rice, jasmine rice, rice noodles, bok choy, broccoli, carrots, mushrooms, shiitake mushrooms, onions, garlic, ginger, sesame oil, rice vinegar, chili flakes, cilantro, scallions, cashews"

Output: 
{'source_column': 'ingredients', 'text': '1 lb. firm tofu, 1/4 c. soy sauce, 4 Tbsp. oil, divided, 2 Tbsp. vinegar, 1/2 c. peas, 4 c. broccoli flowerets, 1 Tbsp. oil, 1 c. fresh mushrooms, sliced, 1/3 c. flour, 1/3 c. oil, 3 c. reserved cooking water, 1/2 c. fresh chopped parsley, 1 1/2 tsp. salt, 1/2 tsp. garlic powder, 1/8 tsp. cayenne', 'title': 'Spaghetti Primavera', 'ingredients': '1 lb. firm tofu, 1/4 c. soy sauce, 4 Tbsp. oil, divided, 2 Tbsp. vinegar, 1/2 c. peas, 4 c. broccoli flowerets, 1 Tbsp. oil, 1 c. fresh mushrooms, sliced, 1/3 c. flour, 1/3 c. oil, 3 c. reserved cooking water, 1/2 c. fresh chopped parsley, 1 1/2 tsp. salt, 1/2 tsp. garlic powder, 1/8 tsp. cayenne', 'directions': 'Cut tofu into pieces 2 x 1/2 x 1/8-inch., Marinate for 2 hours in bowl with soy sauce, 2 tablespoons oil and vinegar., Brown the tofu lightly in leftover marinade plus 2 tablespoons oil; set aside., In boiling water, cook until tender broccoli, fresh or frozen., Reserve water., You may substitute 2 cups broccoli and 2 cups asparagus for the 4 cups broccoli.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 block tofu, 3 stalks green onions, chopped, 1 medium carrot, grated, 3/4 c. mayonnaise, 1 can cream of mushroom soup, 3 to 4 shiitake (dried mushrooms), soaked and chopped, 1 can water chestnuts, chopped, 2 eggs, 1 can tuna, drained', 'title': 'Tofu Casserole', 'ingredients': '1 block tofu, 3 stalks green onions, chopped, 1 medium carrot, grated, 3/4 c. mayonnaise, 1 can cream of mushroom soup, 3 to 4 shiitake (dried mushrooms), soaked and chopped, 1 can water chestnuts, chopped, 2 eggs, 1 can tuna, drained', 'directions': 'Cut tofu into quarters and boil in salted water for 20 minutes., Drain and crumble., Combine with the other ingredients and bake for 40 minutes at 350\.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 c. rice, uncooked, 1 small can mushrooms, 1 envelope onion soup mix, 1 stick margarine, sliced, 2 c. boiling water, 2 tsp. soy sauce, 1 tsp. oregano', 'title': 'Oriental Rice', 'ingredients': '1 c. rice, uncooked, 1 small can mushrooms, 1 envelope onion soup mix, 1 stick margarine, sliced, 2 c. boiling water, 2 tsp. soy sauce, 1 tsp. oregano', 'directions': 'Combine all ingredients., Put into greased casserole., Bake for 1 hour.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1/2 lb. tofu, cut into 1/2-inch cubes (firm or extra firm is best), 3 Tbsp. soy sauce, 1 tsp. vinegar, 1/4 tsp. cumin, 2 cloves garlic, minced, 1/2 tsp. grated ginger, 3 Tbsp. peanut or salad oil, 1 large carrot, chopped, 2 c. chopped broccoli, 1 c. sliced mushrooms, 1/2 c. sliced green onions, 3 Tbsp. cilantro (optional), use any vegetables that are your favorites, crispy noodles', 'title': 'Stir-Fry Vegetables With Tofu', 'ingredients': '1/2 lb. tofu, cut into 1/2-inch cubes (firm or extra firm is best), 3 Tbsp. soy sauce, 1 tsp. vinegar, 1/4 tsp. cumin, 2 cloves garlic, minced, 1/2 tsp. grated ginger, 3 Tbsp. peanut or salad oil, 1 large carrot, chopped, 2 c. chopped broccoli, 1 c. sliced mushrooms, 1/2 c. sliced green onions, 3 Tbsp. cilantro (optional), use any vegetables that are your favorites, crispy noodles', 'directions': 'Place tofu in shallow bowl., Mix soy, vinegar, cumin, garlic and ginger; drizzle this over tofu and set aside., Heat oil in wok or large pan over high heat., Add carrot and stir-fry 1 minute. Add broccoli and cook 2 minutes, then add mushrooms and onions. Reduce heat to medium., Add tofu mixture and cook, stirring gently until heated through but vegetables are still crisp., Garnish with cilantro., Spoon over hot rice and pass the soy sauce and add crispy noodles on top.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. extra firm tofu, 1/2 c. Sake or white wine, 2 Tbsp. sesame oil or vegetable oil, 4 Tbsp. soy sauce or to taste, 2 cloves minced garlic, 2 tsp. ginger juice (optional), 1 onion, thinly sliced, 1 red pepper, diced, 2 c. broccoli florets or Chinese cabbage, 8 oz. snow peas, 1/4 c. roasted cashews or almonds', 'title': 'Stir-Fry Tofu And Veggies', 'ingredients': '1 lb. extra firm tofu, 1/2 c. Sake or white wine, 2 Tbsp. sesame oil or vegetable oil, 4 Tbsp. soy sauce or to taste, 2 cloves minced garlic, 2 tsp. ginger juice (optional), 1 onion, thinly sliced, 1 red pepper, diced, 2 c. broccoli florets or Chinese cabbage, 8 oz. snow peas, 1/4 c. roasted cashews or almonds', 'directions': 'Cut tofu into 1-inch cubes., In wok or skillet, bring soy sauce, garlic, ginger and wine to a boil., Add tofu cubes, cover and simmer for 5 to 10 minutes., Remove tofu and remaining broth from wok or skillet., Heat oil in wok and stir-fry vegetables over medium heat for 3 to 5 minutes., Add remaining broth, tofu and cashews or almonds., Heat and serve.'}

Vector Search Query 2:
"chicken breast, shrimp, brown rice, jasmine rice, rice noodles, bok choy, broccoli, carrots, mushrooms, shiitake mushrooms, onions, garlic, ginger, sesame oil, rice vinegar, chili flakes, cilantro, scallions, cashews"

Output: 
{'source_column': 'ingredients', 'text': 'onion, peppers, mushrooms, olive oil, oleo, peeled, cooked shrimp, fresh parsley, lemon pepper, garlic powder, low-fat milk, 1 heaping Tbsp. plain yogurt, cooked rice', 'title': 'Shrimp And Rice', 'ingredients': 'onion, peppers, mushrooms, olive oil, oleo, peeled, cooked shrimp, fresh parsley, lemon pepper, garlic powder, low-fat milk, 1 heaping Tbsp. plain yogurt, cooked rice', 'directions': 'Saute onion, peppers and mushrooms in olive and oleo combination., Add shrimp, parsley, lemon pepper and garlic powder. Add a bit of low-fat milk and yogurt., Serve over cooked rice.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'zucchini, mushrooms, carrots, broccoli, celery, cherry tomatoes, garlic salt, soy sauce, tarragon, bread crumbs, butter, fish, cheese', 'title': 'Dieters Fish', 'ingredients': 'zucchini, mushrooms, carrots, broccoli, celery, cherry tomatoes, garlic salt, soy sauce, tarragon, bread crumbs, butter, fish, cheese', 'directions': 'Place fish in pan., Surround with vegetables., Pat fish with butter., Sprinkle on garlic salt, soy sauce, tarragon, bread crumbs and grated cheese., Seal whole dish with foil., Bake 30 minutes at 450\.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'chicken, veal, shrimp, fish or lobster, eggs, flour, oil, margarine, lemon juice, white wine, salt and pepper, mushrooms', 'title': 'Francais With Chicken, Veal, Shrimp, Fish Or Lobster', 'ingredients': 'chicken, veal, shrimp, fish or lobster, eggs, flour, oil, margarine, lemon juice, white wine, salt and pepper, mushrooms', 'directions': 'Pound and wash meat., Add a little water to eggs and whisk. Flour meat., Put oil in pan and heat until very hot., Add meat and cook until light brown., Empty oil from pan., Put, margarine, lemon juice, white wine, along with meat already in pan., Salt and pepper., Add mushrooms., Fix rice with bay leaf, lemon, margarine, chicken flavor and bouillon cube.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'boneless chicken breast, broccoli (stems and florets), carrots, celery, cooking oil or olive oil, green pepper, mushrooms, onion, purchased pkg. shrimp and broccoli mix, seasonings, soy sauce, butter or margarine, pepper, rice, salt', 'title': 'Chicken And Vegetable Stir-Fry Over Rice', 'ingredients': 'boneless chicken breast, broccoli (stems and florets), carrots, celery, cooking oil or olive oil, green pepper, mushrooms, onion, purchased pkg. shrimp and broccoli mix, seasonings, soy sauce, butter or margarine, pepper, rice, salt', 'directions': 'Chicken and Vegetables:, Prepare vegetables and meat., Chop vegetables in bite-size pieces., Slice chicken breast in bite-size pieces or purchase chicken stir-fry.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'broccoli, cauliflower, tomatoes, green onions, mushrooms, Italian dressing, black olives, bell peppers, celery, radishes, carrots', 'title': 'Christmas Salad(Or No Lettuce Salad)  ', 'ingredients': 'broccoli, cauliflower, tomatoes, green onions, mushrooms, Italian dressing, black olives, bell peppers, celery, radishes, carrots', 'directions': 'Cut or chop all vegetables., Combine all vegetables in bowl. Pour Italian dressing over salad and let set overnight in refrigerator., You can use any fresh vegetables in this salad.'}"""

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\3614672339.py:1: SyntaxWarning: invalid escape sequence '\.'
  agentic_rag_outputs["input7"] = """**Breakfast Recipe: Chicken and Mushroom Stir-Fry**


In [36]:
print(agentic_rag_mp(inputs["input8"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Canned tuna, canned beans (kidney, black, chickpeas), lentils, rice, pasta, potatoes, onions, carrots, frozen vegetables (peas, corn, broccoli), eggs, peanut butter, oats, whole wheat bread, canned tomatoes, olive oil, spices (cumin, chili powder, oregano)
Dietary information: None
Allergy information: Peanut
Daily protein target: 100g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: The user has provided a list of ingredients, no dietary restrictions, a peanut allergy, and a daily protein target of 100g. To create a complete weekly meal plan, I need to generate three distinct recipes for breakfast, lunch, and dinner, and one recipe for a mid-day snack. Since the user has a peanut allergy, I will avoid using peanut butter in the recipes. First, I will use the Vector Search Tool to query the vector database for recipes that include the provided ingredients and meet the daily protein target.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"high protein recipes using canned tuna, canned beans, lentils, rice, pasta, potatoes, onions, carrots, frozen vegetables, eggs, oats, whole wheat bread, canned tomatoes, olive oil, spices\"}"
## Tool Output: 
{'source_colu

In [37]:
agentic_rag_outputs["input8"] = """**Weekly Meal Plan**

**Breakfast Recipe: Egg and Lentil Scramble**

Ingredients:
- 1 dozen eggs
- 1 cup cooked lentils
- 1 medium onion, chopped
- 2 cloves garlic, minced
- 1 medium carrot, peeled and grated
- 1 teaspoon cumin
- Salt and pepper to taste

Instructions:
1. In a large bowl, whisk together eggs and a pinch of salt.
2. Heat a large skillet over medium heat and add the chopped onion, garlic, and grated carrot. Cook until the vegetables are tender.
3. Add the cooked lentils to the skillet and stir to combine.
4. Pour the whisked eggs over the lentil mixture and scramble until the eggs are cooked through.
5. Season with cumin, salt, and pepper to taste.

Nutritional Information (per serving):
- Calories: 250
- Protein: 20g
- Fat: 10g
- Carbohydrates: 20g

**Lunch Recipe: Tuna and White Bean Salad**

Ingredients:
- 3 cans tuna
- 1 can cannellini beans, drained and rinsed
- 1 medium onion, chopped
- 2 cloves garlic, minced
- 1 medium carrot, peeled and grated
- 2 tablespoons olive oil
- 1 teaspoon oregano
- Salt and pepper to taste

Instructions:
1. In a large bowl, combine the canned tuna, cannellini beans, chopped onion, garlic, and grated carrot.
2. In a small bowl, whisk together olive oil and oregano.
3. Pour the dressing over the tuna mixture and toss to combine.
4. Season with salt and pepper to taste.

Nutritional Information (per serving):
- Calories: 300
- Protein: 35g
- Fat: 15g
- Carbohydrates: 20g

**Dinner Recipe: Chili with Kidney Beans and Vegetables**

Ingredients:
- 1 can kidney beans, drained and rinsed
- 1 medium onion, chopped
- 2 cloves garlic, minced
- 1 medium carrot, peeled and grated
- 1 can diced tomatoes
- 1 cup frozen vegetables (peas, corn, broccoli)
- 1 teaspoon chili powder
- 1 teaspoon cumin
- Salt and pepper to taste

Instructions:
1. In a large pot, heat a couple of tablespoons of olive oil over medium heat.
2. Add the chopped onion, garlic, and grated carrot to the pot and cook until the vegetables are tender.
3. Add the kidney beans, diced tomatoes, frozen vegetables, chili powder, and cumin to the pot.
4. Stir to combine and bring to a simmer.
5. Reduce heat to low and let cook for 20-30 minutes or until the flavors have melded together.
6. Season with salt and pepper to taste.

Nutritional Information (per serving):
- Calories: 400
- Protein: 25g
- Fat: 10g
- Carbohydrates: 60g

**Mid-Day Snack Recipe: Oat and Egg Bites**

Ingredients:
- 1 cup rolled oats
- 4 eggs
- 1/2 cup grated carrot
- 1/2 cup chopped onion
- 1/4 cup chopped fresh parsley
- Salt and pepper to taste

Instructions:
1. Preheat the oven to 375°F (190°C).
2. In a large bowl, whisk together eggs and a pinch of salt.
3. Add the grated carrot, chopped onion, and chopped parsley to the bowl and stir to combine.
4. Add the rolled oats to the bowl and stir until the mixture is well combined.
5. Spoon the mixture into a greased mini muffin tin.
6. Bake for 15-20 minutes or until the edges are lightly golden brown.

Nutritional Information (per serving):
- Calories: 150
- Protein: 10g
- Fat: 5g
- Carbohydrates: 20g

This meal plan provides approximately 100g of protein per day, spread across three meals and one snack. The recipes include a variety of ingredients and are easy to prepare. The meal plan is also customizable to suit individual tastes and dietary needs.

Vector Search Query:
"high protein recipes using canned tuna, canned beans, lentils, rice, pasta, potatoes, onions, carrots, frozen vegetables, eggs, oats, whole wheat bread, canned tomatoes, olive oil, spices\"}"

Output: 
{'source_column': 'ingredients', 'text': '3 cans tuna, 1 can cream of chicken soup, 1 can tomatoes, 1 pkg. frozen peas, 3 pkg. macaroni and cheese, garlic, salt and pepper, 1/2 c. extra milk', 'title': "Judy'S Tuna Surprise", 'ingredients': '3 cans tuna, 1 can cream of chicken soup, 1 can tomatoes, 1 pkg. frozen peas, 3 pkg. macaroni and cheese, garlic, salt and pepper, 1/2 c. extra milk', 'directions': 'Prepare macaroni and cheese as box directions says., Add tuna, tomatoes, peas, can of cream of chicken soup, milk, garlic, salt and pepper., Mix well., Put in casserole dish and bake at 400 for 10 to 20 minutes, until brown on top.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 can cream of chicken soup, 1/2 can milk, 1 can tuna or 2 small, 2 c. egg noodles or elbows, butter, bread crumbs, pkg. frozen peas and carrots, cooked', 'title': 'Tuna Casserole', 'ingredients': '1 can cream of chicken soup, 1/2 can milk, 1 can tuna or 2 small, 2 c. egg noodles or elbows, butter, bread crumbs, pkg. frozen peas and carrots, cooked', 'directions': 'Cook noodles and drain., Drain tuna., Cook peas and carrots. In large bowl, mix soup and milk., Add tuna, noodles and vegetables., Put in baking dish., Sprinkle with bread crumbs and put a few pieces of butter on top., Bake at 350 about 30 minutes.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 medium can tuna, 1 can cream of mushroom soup, 2 c. cooked noodles, 1 can sweet peas, 1/2 c. grated onion, 1/2 stick margarine, 1 c. milk, 2 c. grated sharp cheese', 'title': 'Tuna Casserole', 'ingredients': '1 medium can tuna, 1 can cream of mushroom soup, 2 c. cooked noodles, 1 can sweet peas, 1/2 c. grated onion, 1/2 stick margarine, 1 c. milk, 2 c. grated sharp cheese', 'directions': 'Mix ingredients together., Add salt, pepper and 1/2 cup of cheese on top, if preferred., Cook at 350 until golden brown.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': 'Ground beef or steak, chopped onion, chopped celery, can of tomatoes, small can V-8 juice, 1 Tbsp. flour, 1 tsp. chili powder, 1 can kidney beans, 1 Tbsp. brown sugar', 'title': 'Chili', 'ingredients': 'Ground beef or steak, chopped onion, chopped celery, can of tomatoes, small can V-8 juice, 1 Tbsp. flour, 1 tsp. chili powder, 1 can kidney beans, 1 Tbsp. brown sugar', 'directions': 'Saute meat, onions and celery. Add tomatoes and V-8 juice. Then add remaining ingredients. Simmer for 2 -3 hours.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '2 c. T.V.P. flakes (texturized vegetable protein; available in health food stores), 2 Tbsp. ketchup, 1 large onion, chopped, 1 green pepper, chopped, 2 cloves garlic, chopped, 2 Tbsp. olive oil, 2 Tbsp. chili powder, 1/2 tsp. cayenne, 2 tsp. cumin, 2 tsp. oregano, 2 (14 oz.) cans diced tomatoes, 1 (40 1/2 oz.) can kidney beans, rinsed, 1 (28 oz.) can crushed tomatoes, 3 c. V-8 juice', 'title': 'Country Chili', 'ingredients': '2 c. T.V.P. flakes (texturized vegetable protein; available in health food stores), 2 Tbsp. ketchup, 1 large onion, chopped, 1 green pepper, chopped, 2 cloves garlic, chopped, 2 Tbsp. olive oil, 2 Tbsp. chili powder, 1/2 tsp. cayenne, 2 tsp. cumin, 2 tsp. oregano, 2 (14 oz.) cans diced tomatoes, 1 (40 1/2 oz.) can kidney beans, rinsed, 1 (28 oz.) can crushed tomatoes, 3 c. V-8 juice', 'directions': 'Pour 2 cups boiling water over T.V.P. flakes and ketchup and let stand for 10 minutes. Heat olive oil in large Dutch oven. Over medium heat, saute onion, green pepper and garlic. Sprinkle chili powder, cayenne, cumin and oregano over T.V.P. mixture. Stir and add to mixture in Dutch oven. Cook a few minutes and stir in canned tomatoes, kidney beans, crushed tomatoes and V-8 juice. Cover and simmer for 30 minutes to an hour. Add salt and pepper to taste.'}"""

In [38]:
print(agentic_rag_mp(inputs["input9"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Turkey breast, lean ground beef, tilapia fillets, egg whites, brown rice, wild rice, quinoa, steel-cut oats, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper.
Dietary information: None
Allergy information: None
Daily protein target: 120g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: I need to create a weekly meal plan for the user, taking into account their ingredients, dietary restrictions, allergies, and daily protein target. To do this, I will first use the Vector Search Tool to find relevant recipes that incorporate the user's ingredients and meet their protein requirements.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"high protein recipes using turkey breast, lean ground beef, tilapia fillets, egg whites, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper\"}"
## Tool Output: 
{'source_column': 'ingredients', 'text': '1 lb. lean ground beef or turkey, 1 small, chopped onion, 1 garlic clove, minced, 1 c. sour 

In [39]:
agentic_rag_outputs["input9"] = """**Weekly Meal Plan**

### Breakfast: Turkey Breast and Sweet Potato Omelette

* 7 servings
* Ingredients:
	+ 1.75 lbs turkey breast, diced (Must be cooked)
	+ 3.5 large sweet potatoes, peeled and diced
	+ 14 large eggs
	+ 1/2 cup chopped onion
	+ 1/2 cup chopped bell peppers
	+ 1 tsp salt
	+ 1/4 tsp black pepper
* Instructions:
	1. Preheat oven to 400°F.
	2. Bake the diced sweet potatoes for 20-25 minutes, or until tender.
	3. In a large bowl, whisk together the eggs, salt, and pepper.
	4. Add the diced turkey breast, chopped onion, and chopped bell peppers to the bowl and stir to combine.
	5. Heat a large non-stick skillet over medium heat.
	6. Pour the egg mixture into the skillet and cook until the eggs start to set.
	7. Add the baked sweet potatoes to the skillet and stir to combine.
	8. Use a spatula to fold the omelette in half and cook for an additional 2-3 minutes.
* Nutritional Information (per serving):
	+ Calories: 320
	+ Protein: 35g
	+ Fat: 10g
	+ Carbohydrates: 20g

### Lunch: Grilled Chicken and Vegetable Stir-Fry

* 7 servings
* Ingredients:
	+ 1.75 lbs tilapia fillets (Must be cooked), cut into small pieces
	+ 3.5 cups mixed vegetables (broccoli, cauliflower, carrots)
	+ 1/2 cup chopped onion
	+ 1/2 cup chopped bell peppers
	+ 2 tbsp olive oil
	+ 1 tsp soy sauce
	+ Salt and pepper to taste
* Instructions:
	1. Preheat grill or grill pan to medium-high heat.
	2. Grill the tilapia fillets for 3-4 minutes per side, or until cooked through.
	3. In a large bowl, toss together the mixed vegetables, chopped onion, and chopped bell peppers.
	4. In a small bowl, whisk together the olive oil and soy sauce.
	5. Pour the oil and soy sauce mixture over the vegetables and toss to combine.
	6. Add the grilled tilapia fillets to the bowl and stir to combine.
	7. Season with salt and pepper to taste.
* Nutritional Information (per serving):
	+ Calories: 300
	+ Protein: 30g
	+ Fat: 10g
	+ Carbohydrates: 20g

### Dinner: Beef and Vegetable Kabobs

* 7 servings
* Ingredients:
	+ 1.75 lbs lean ground beef
	+ 3.5 cups mixed vegetables (asparagus, bell peppers, onions, mushrooms)
	+ 1/4 cup olive oil
	+ 1 tsp salt
	+ 1/4 tsp black pepper
* Instructions:
	1. Preheat grill or grill pan to medium-high heat.
	2. Thread the lean ground beef and mixed vegetables onto skewers.
	3. Brush the skewers with olive oil and season with salt and pepper.
	4. Grill the skewers for 8-10 minutes, or until the beef is cooked through.
	5. Serve hot.
* Nutritional Information (per serving):
	+ Calories: 350
	+ Protein: 25g
	+ Fat: 15g
	+ Carbohydrates: 20g

### Mid-Day Snack: Protein Smoothie

* 7 servings
* Ingredients:
	+ 1.75 cups egg whites
	+ 1 cup frozen spinach
	+ 1 cup frozen berries
	+ 1/2 cup unsweetened almond milk
	+ 1 scoop protein powder (Must be purchased)
* Instructions:
	1. Combine all ingredients in a blender and blend until smooth.
	2. Pour into a glass and serve immediately.
* Nutritional Information (per serving):
	+ Calories: 150
	+ Protein: 20g
	+ Fat: 0g
	+ Carbohydrates: 10g

Note: The nutritional information is an estimate and may vary based on specific ingredients and portion sizes. It is always a good idea to consult with a healthcare professional or registered dietitian for personalized nutrition advice.

Vector Search Query:
"high protein recipes using turkey breast, lean ground beef, tilapia fillets, egg whites, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper"

Output: 
{'source_column': 'ingredients', 'text': '1 lb. lean ground beef or turkey, 1 small, chopped onion, 1 garlic clove, minced, 1 c. sour cream, 1/3 c. hot chunky salsa, 1 tsp. oregano, 1/8 tsp. chili powder, 1/2 tsp. salt, 4 baked potatoes', 'title': 'Mexican Hot Potatoes', 'ingredients': '1 lb. lean ground beef or turkey, 1 small, chopped onion, 1 garlic clove, minced, 1 c. sour cream, 1/3 c. hot chunky salsa, 1 tsp. oregano, 1/8 tsp. chili powder, 1/2 tsp. salt, 4 baked potatoes', 'directions': 'In a large skillet, brown beef with onion and garlic; drain. Stir in sour cream, salsa, oregano, chili powder and salt., Serve over hot, baked potato and with guacamole on the side.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 c. cooked meat, finely diced (pork, beef, crab, shrimp, ham or chicken), 2 c. bean sprouts, 1 c. celery, finely diced, 1/2 c. onions, finely diced, 3 water chestnuts, finely diced (optional), 6 to 7 eggs, 1/2 tsp. salt, 1/4 tsp. m.s.g., 1 tsp. soy sauce, 2 Tbsp. oil (peanut oil)', 'title': 'Egg Foo Yong', 'ingredients': '1 c. cooked meat, finely diced (pork, beef, crab, shrimp, ham or chicken), 2 c. bean sprouts, 1 c. celery, finely diced, 1/2 c. onions, finely diced, 3 water chestnuts, finely diced (optional), 6 to 7 eggs, 1/2 tsp. salt, 1/4 tsp. m.s.g., 1 tsp. soy sauce, 2 Tbsp. oil (peanut oil)', 'directions': 'Heat 1 tablespoon oil in frypan., Add meat, onions and celery. Stir-fry for 1 minute., Add bean sprouts and seasonings., Cover and cook 1 minute., Remove from pan and place in a large mixing bowl., Add eggs and fold in., Add 1 tablespoon oil to pan and heat thoroughly., Spoon mixture-like pancake size batter onto pan and fry gently over low heat until eggs are set., Turn over and brown the other side., Repeat until all the batter is used., Serve with rice and a brown gravy., Serves 4.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. butternut squash, 3 tart apples, 1 medium onion, 1/4 tsp. rosemary, 1/4 tsp. marjoram, 6 to 7 c. chicken broth, 1 tsp. salt, 1/4 tsp. fresh ground pepper, 1/2 c. heavy cream, 1 Tbsp. butter, 1 medium onion, minced, 1/2 tsp. thyme, 16 oz. tomatoes (including juice), chopped, 2 garlic cloves, minced, 2 c. beef broth (or more), 2 c. cooked chickpeas (drained if in can and rinsed), 1/2 c. small pasta, cooked, ground pepper, 1/4 c. grated Parmesan', 'title': 'Apple And Butternut Squash Soup', 'ingredients': '1 lb. butternut squash, 3 tart apples, 1 medium onion, 1/4 tsp. rosemary, 1/4 tsp. marjoram, 6 to 7 c. chicken broth, 1 tsp. salt, 1/4 tsp. fresh ground pepper, 1/2 c. heavy cream, 1 Tbsp. butter, 1 medium onion, minced, 1/2 tsp. thyme, 16 oz. tomatoes (including juice), chopped, 2 garlic cloves, minced, 2 c. beef broth (or more), 2 c. cooked chickpeas (drained if in can and rinsed), 1/2 c. small pasta, cooked, ground pepper, 1/4 c. grated Parmesan', 'directions': 'Melt butter; cook onion with thyme for 5 minutes., Add tomatoes, juice and garlic., Simmer for 15 minutes., Add broth and chickpeas., Simmer for another 15 minutes., Add cooked pasta and pepper., Cook until warm., Serve with Parmesan cheese on top.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. ground beef, 1/2 c. diced celery, 1 c. cream of mushroom soup, 1 c. cooked rice, dash of garlic, 1 c. frozen peas, 1 medium onion, diced, 1 can water chestnuts, 1 1/2 tsp. soy sauce, 1/2 c. chicken broth, 2 c. chow mein noodles', 'title': 'Chinese Casserole', 'ingredients': '1 lb. ground beef, 1/2 c. diced celery, 1 c. cream of mushroom soup, 1 c. cooked rice, dash of garlic, 1 c. frozen peas, 1 medium onion, diced, 1 can water chestnuts, 1 1/2 tsp. soy sauce, 1/2 c. chicken broth, 2 c. chow mein noodles', 'directions': 'Combine, ground, beef, celery and onion; microwave on High for 5 minutes, stir once., Combine remaining ingredients, except frozen, peas and chow mein noodles., Microwave on High for 3 minutes, covered., Lower power to 80 and microwave for 6, to 8 minutes., Add frozen peas. Sprinkle chow mein noodles over top of casserole. Microwave 1 minute on High, uncovered.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 small onion, minced, 1 clove garlic, minced, 3 Tbsp. butter, 3 Tbsp. olive oil, 1 lb. ground beef, 1/2 lb. bulk pork sausage, 1 (No. 2 1/2) can tomatoes (3 1/2 c.), 1 (No. 2) can whole kernel corn (2 1/2 c.), 2 tsp. salt, 2 tsp. chili powder, 20 to 24 pitted ripe olives, 1 c. corn meal, 2 eggs, well beaten, 1 1/2 c. grated American cheese', 'title': 'Fiesta Tamale Pie', 'ingredients': '1 small onion, minced, 1 clove garlic, minced, 3 Tbsp. butter, 3 Tbsp. olive oil, 1 lb. ground beef, 1/2 lb. bulk pork sausage, 1 (No. 2 1/2) can tomatoes (3 1/2 c.), 1 (No. 2) can whole kernel corn (2 1/2 c.), 2 tsp. salt, 2 tsp. chili powder, 20 to 24 pitted ripe olives, 1 c. corn meal, 2 eggs, well beaten, 1 1/2 c. grated American cheese', 'directions': 'Saute onion and garlic in butter and oil until onion is yellow. Add meat., Cook until brown.'}"""

In [40]:
print(agentic_rag_mp(inputs["input10"]))

Overriding of current TracerProvider is not allowed


# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Task: Create a complete weekly meal plan based on the user's input.

Ingredients: Turkey breast, lean ground beef, tilapia fillets, egg whites, brown rice, wild rice, quinoa, steel-cut oats, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper.
Dietary information: None
Allergy information: None
Daily protein target: 95g


C:\Users\ACER\AppData\Local\Temp\ipykernel_59396\635197441.py:17: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant.search(




# Agent: You are a professional meal prep specialist and nutritionist with expertise in creating efficient and customized weekly meal plans using the help of external tools.
## Thought: Thought: To create a complete weekly meal plan, I need to analyze the user's input to understand the ingredients they have, their dietary restrictions, allergies, and daily protein target. The user has provided a list of ingredients, no dietary restrictions, no allergies, and a daily protein target of 95g.
## Using tool: Vector Search Tool
## Tool Input: 
"{\"query\": \"high protein recipes using turkey breast, lean ground beef, tilapia fillets, egg whites, brown rice, wild rice, quinoa, steel-cut oats, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper\"}"
## Tool Output: 
{'source_column': 'ingredients', 'text': '3 c. long grain rice, 1 c. pigeon peas (granules

In [41]:
agentic_rag_outputs["input10"] = """**Breakfast Recipe: Turkey and Sweet Potato Hash**

* 1 lb turkey breast, diced
* 2 large sweet potatoes, peeled and diced
* 1 medium onion, diced
* 2 cloves garlic, minced
* 1 cup spinach
* 1/2 cup steel-cut oats
* 1 tablespoon olive oil
* Salt and pepper to taste

Total quantity of ingredients needed for 7 servings:
* 7 lbs turkey breast
* 14 large sweet potatoes
* 7 medium onions
* 14 cloves garlic
* 7 cups spinach
* 3.5 cups steel-cut oats
* 7 tablespoons olive oil

Instructions:
1. Preheat oven to 400°F.
2. In a large bowl, toss sweet potatoes with 2 tablespoons olive oil, salt, and pepper. Spread on a baking sheet and roast for 20-25 minutes, or until tender.
3. In a large skillet, heat 1 tablespoon olive oil over medium-high heat. Add diced turkey breast and cook until browned, about 5-7 minutes.
4. Add diced onion and minced garlic to the skillet and cook until the onion is translucent.
5. Stir in spinach and cook until wilted.
6. In a large bowl, combine cooked turkey mixture, roasted sweet potatoes, and steel-cut oats.
7. Divide into 7 portions and serve.

Nutritional information per serving:
* Calories: 350
* Protein: 35g
* Fat: 10g
* Carbohydrates: 30g

**Lunch Recipe: Quinoa and Blackened Tilapia**

* 1 lb tilapia fillets
* 1 cup quinoa
* 2 cups water
* 1 medium onion, diced

Vector Search Query:
"high protein recipes using turkey breast, lean ground beef, tilapia fillets, egg whites, brown rice, wild rice, quinoa, steel-cut oats, sweet potatoes, asparagus, bell peppers, broccoli, spinach, kale, cauliflower, mushrooms, onions, garlic, ginger, lemon, lime, olive oil, apple cider vinegar, cinnamon, turmeric, black pepper"

Output: 
{'source_column': 'ingredients', 'text': '3 c. long grain rice, 1 c. pigeon peas (granules), 2 cubes chicken bouillon, 1/4 c. green stuffed olives, 1 medium onion, chopped, 1/4 c. oil, 1 small can red bell peppers, 1 can tomato sauce, 5 1/2 c. boiling water, 1 1/2 lb. pork meat, cut into 1-inch cubes, 2 cloves garlic, minced, 1/4 c. chopped cilantro, 2 pkg. Sazon (found in international food section), salt and pepper', 'title': 'Rice And Pigeon Peas With Pork', 'ingredients': '3 c. long grain rice, 1 c. pigeon peas (granules), 2 cubes chicken bouillon, 1/4 c. green stuffed olives, 1 medium onion, chopped, 1/4 c. oil, 1 small can red bell peppers, 1 can tomato sauce, 5 1/2 c. boiling water, 1 1/2 lb. pork meat, cut into 1-inch cubes, 2 cloves garlic, minced, 1/4 c. chopped cilantro, 2 pkg. Sazon (found in international food section), salt and pepper', 'directions': 'In a large saucepan, add oil and brown meat., Add onion, garlic and cilantro., Let it simmer for about 2 minutes., Add red bell peppers, green olives, chicken bouillon cubes, tomato sauce, Sazon, pigeon peas, salt and pepper., Let simmer for about 5 minutes., Add the rice and water; mix real well., Bring to a boil and let the water evaporate., Mix it again., Cover and cook over a low flame for about 15 to 20 minutes.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. lean ground beef or turkey, 1 small, chopped onion, 1 garlic clove, minced, 1 c. sour cream, 1/3 c. hot chunky salsa, 1 tsp. oregano, 1/8 tsp. chili powder, 1/2 tsp. salt, 4 baked potatoes', 'title': 'Mexican Hot Potatoes', 'ingredients': '1 lb. lean ground beef or turkey, 1 small, chopped onion, 1 garlic clove, minced, 1 c. sour cream, 1/3 c. hot chunky salsa, 1 tsp. oregano, 1/8 tsp. chili powder, 1/2 tsp. salt, 4 baked potatoes', 'directions': 'In a large skillet, brown beef with onion and garlic; drain. Stir in sour cream, salsa, oregano, chili powder and salt., Serve over hot, baked potato and with guacamole on the side.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 lb. ground beef, 1/2 c. diced celery, 1 c. cream of mushroom soup, 1 c. cooked rice, dash of garlic, 1 c. frozen peas, 1 medium onion, diced, 1 can water chestnuts, 1 1/2 tsp. soy sauce, 1/2 c. chicken broth, 2 c. chow mein noodles', 'title': 'Chinese Casserole', 'ingredients': '1 lb. ground beef, 1/2 c. diced celery, 1 c. cream of mushroom soup, 1 c. cooked rice, dash of garlic, 1 c. frozen peas, 1 medium onion, diced, 1 can water chestnuts, 1 1/2 tsp. soy sauce, 1/2 c. chicken broth, 2 c. chow mein noodles', 'directions': 'Combine, ground, beef, celery and onion; microwave on High for 5 minutes, stir once., Combine remaining ingredients, except frozen, peas and chow mein noodles., Microwave on High for 3 minutes, covered., Lower power to 80 and microwave for 6, to 8 minutes., Add frozen peas. Sprinkle chow mein noodles over top of casserole. Microwave 1 minute on High, uncovered.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '1 c. wild rice, 2 lb. lean ground beef, 1 medium onion, diced, 1/2 c. celery, diced, 2 Tbsp. low-salt soy sauce, 1 can chicken rice soup, 1 can cream of mushroom soup, 1 can mushrooms (with juice), 1 c. slivered almonds', 'title': 'Wild Rice Casserole', 'ingredients': '1 c. wild rice, 2 lb. lean ground beef, 1 medium onion, diced, 1/2 c. celery, diced, 2 Tbsp. low-salt soy sauce, 1 can chicken rice soup, 1 can cream of mushroom soup, 1 can mushrooms (with juice), 1 c. slivered almonds', 'directions': 'Cook wild rice in as little water as possible, until tender. Brown beef and onion; drain well., Blend all ingredients except almonds., Bake in greased casserole at 325 for 1 hour.'}Retrieved recipes:

{'source_column': 'ingredients', 'text': '2 lb. beef, cubed, 4 large carrots, chunked, 2 medium onions, chunked, 2 celery ribs, chunked, 2 medium parsnips, chunked, 2 medium potatoes, cubed, 1 garlic clove, minced, 1 can stewed tomatoes, 1 1/2 c. beef broth, 1 (8 oz.) can tomato sauce, 1/2 c. quick cook tapioca, 1 tsp. instant coffee, 1/2 tsp. thyme, 1/2 tsp. oregano, salt and pepper', 'title': 'Easy Oven Stew', 'ingredients': '2 lb. beef, cubed, 4 large carrots, chunked, 2 medium onions, chunked, 2 celery ribs, chunked, 2 medium parsnips, chunked, 2 medium potatoes, cubed, 1 garlic clove, minced, 1 can stewed tomatoes, 1 1/2 c. beef broth, 1 (8 oz.) can tomato sauce, 1/2 c. quick cook tapioca, 1 tsp. instant coffee, 1/2 tsp. thyme, 1/2 tsp. oregano, salt and pepper', 'directions': 'Combine all ingredients in a 5-quart Dutch oven., Cover and bake at 300 for 2 1/2 to 3 hours, stirring once each hour., Makes 8 servings.'}"""

In [42]:
# Printing the outputs of the Agentic RAG System
print(json.dumps(agentic_rag_outputs, indent=4))

{
    "input1": "# Weekly Meal Plan\n\n## Breakfast: Grilled Chicken and Vegetable Skillet\n### Ingredients:\n- 7 cups mixed vegetables (bell peppers, onions, zucchini)\n- 7 cups cooked chicken thighs\n- 3.5 cups cooked quinoa\n- 7 tablespoons olive oil\n- Salt and pepper to taste\n- Fresh parsley, chopped (optional)\n### Instructions:\n1. Preheat the grill or grill pan to medium-high heat.\n2. Season the chicken thighs with salt and pepper.\n3. Grill the chicken for 5-6 minutes per side or until cooked through.\n4. Meanwhile, heat the olive oil in a large skillet over medium heat.\n5. Add the mixed vegetables and cook until tender, about 5 minutes.\n6. Serve the grilled chicken with the vegetable skillet and quinoa.\n### Nutritional Information (per serving):\n- Calories: 420\n- Protein: 36g\n- Fats: 20g\n- Carbohydrates: 20g\n\n## Lunch: Baked Salmon with Roasted Vegetables\n### Ingredients:\n- 7 salmon fillets (6 oz each)\n- 7 cups mixed vegetables (eggplant, tomatoes, red onion)\n-

### Individually running each output with the LLM judge to keep intervals to not hit rate limits and printing the results 

In [43]:
org_inp = inputs["input1"]
agentic_out = agentic_rag_outputs["input1"]
standard_out = standard_rag_outputs["input1"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System's vector search query is highly relevant to the user's input, incorporating key ingredients, dietary restrictions, and the daily protein target. The retrieved recipes, although not all directly usable, demonstrate a good understanding of the context. The generated meal plan closely meets the daily protein target and adheres to the dietary restrictions, utilizing most of the provided ingredients efficiently. Minor points are deducted for not explicitly tagging all non-listed ingredients as "Must be purchased" and the potential for slight nutritional discrepancies.

Standard RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 7/10
Feedback: The Standard RAG System's vector search query, while comprehensive, does not yield as relevant results as the Agentic RAG System, with many retrieved recipes containing dairy or gluten. The generated meal plan falls short

In [44]:
org_inp = inputs["input2"]
agentic_out = agentic_rag_outputs["input2"]
standard_out = standard_rag_outputs["input2"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System demonstrated a high level of context relevance by retrieving recipes that closely match the user's input, such as high-protein breakfast recipes and salmon with quinoa. The system also showed high answer accuracy by providing a meal plan that meets the daily protein target of 120g and utilizes the provided ingredients effectively. However, the system's context relevance score is not perfect due to the retrieval of some recipes that may not be directly relevant to the user's input, such as the "Low Calorie Island Dressing" recipe.

Standard RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 8/10
Feedback: The Standard RAG System demonstrated a moderate level of context relevance by retrieving some recipes that match the user's input, but also retrieving some recipes that may not be directly relevant, such as the "Chicken With Prosciutto" recipe. The system

In [45]:
org_inp = inputs["input3"]
agentic_out = agentic_rag_outputs["input3"]
standard_out = standard_rag_outputs["input3"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System provided a meal plan that closely meets the user's daily protein target of 100g, with a total daily protein intake of 103g. The system also effectively utilized the ingredients provided by the user, minimizing the need for additional purchases. However, the vector search query could be improved to more specifically target recipes that meet the user's dietary restrictions and allergies.

Standard RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 7/10
Feedback: The Standard RAG System provided a meal plan that does not fully meet the user's daily protein target, with a total daily protein intake of 65g. The system also failed to effectively utilize the ingredients provided by the user, requiring additional purchases such as honey and coconut oil. Furthermore, the vector search query retrieved recipes that contain soy and peanuts, which are allergens specif

In [46]:
org_inp = inputs["input4"]
agentic_out = agentic_rag_outputs["input4"]
standard_out = standard_rag_outputs["input4"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System's output is highly relevant to the context, utilizing most of the provided ingredients and meeting the daily protein target of 105g. The system's ability to generate a meal plan with specific recipes, including nutritional information, demonstrates a high level of accuracy. However, the system's vector search query could be improved to retrieve more relevant recipes.

Standard RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 7/10
Feedback: The Standard RAG System's output is somewhat relevant to the context, but it does not fully utilize the provided ingredients and relies on additional ingredients not listed in the user's input. The system's meal plan meets the daily protein target, but the recipes lack variety and the nutritional information is not as comprehensive as the Agentic RAG System's output. The system's vector search query retrieves recipes 

In [47]:
org_inp = inputs["input5"]
agentic_out = agentic_rag_outputs["input5"]
standard_out = standard_rag_outputs["input5"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System demonstrates high context relevance by retrieving recipes that, although not directly matching the user's query, allowed the system to create custom recipes meeting the user's dietary needs and preferences. The system accurately hits the daily protein target of 95g and incorporates the majority of the provided ingredients, minimizing the need for additional purchases. However, it requires the purchase of several items such as whole wheat bread, smoked salmon, salmon fillets, Greek yogurt, granola, and honey, which might not be considered minimal. The system also demonstrates a good understanding of the pescatarian diet by excluding non-fish meat sources.

Standard RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 8/10
Feedback: The Standard RAG System shows lower context relevance due to the retrieval of recipes that are less relevant to the user's input

In [48]:
org_inp = inputs["input6"]
agentic_out = agentic_rag_outputs["input6"]
standard_out = standard_rag_outputs["input6"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System provided a meal plan that closely meets the daily protein target of 85g, with a total protein intake of 95g (30g from breakfast, 35g from lunch, 20g from dinner, and 10g from the snack). The system utilized the provided ingredients efficiently, with some additional ingredients tagged as "Must be purchased". The nutritional information for each recipe is accurate, and the system demonstrates a good understanding of the user's requirements. However, the context relevance score is not perfect due to the retrieval of some irrelevant recipes in the vector search results.

Standard RAG System:
- Context Relevance score: 2/10
- Answer Accuracy score: 6/10
Feedback: The Standard RAG System provided a meal plan that does not closely meet the daily protein target of 85g, with a total protein intake of 90g (15g from breakfast, 40g from lunch, 30g from dinner, and 5g from the snack). 

In [49]:
org_inp = inputs["input7"]
agentic_out = agentic_rag_outputs["input7"]
standard_out = standard_rag_outputs["input7"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 4/10
Feedback: The Agentic RAG System's context relevance score is 6 due to the retrieved documents being somewhat related to the user's input, but not entirely relevant to the task. The system failed to fully consider the dietary restrictions and allergy information provided by the user. The answer accuracy score is 4 because the system's output does not accurately satisfy the constraints given in the task, such as hitting the daily protein intake target and excluding ingredients that the user is allergic to. The system also failed to minimize the usage of ingredients not listed in the user's ingredients list.

Standard RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 8/10
Feedback: The Standard RAG System's context relevance score is 8 due to the retrieved documents being more relevant to the user's input and task. The system successfully considered the dietary restrictions and allergy in

In [50]:
org_inp = inputs["input8"]
agentic_out = agentic_rag_outputs["input8"]
standard_out = standard_rag_outputs["input8"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Agentic RAG System's output is highly relevant to the user's input and task, as it provides a comprehensive meal plan that incorporates most of the given ingredients and meets the daily protein target. The recipes are well-structured, and the nutritional information is accurately calculated. However, the system could improve by providing more variety in the meal plan and considering potential allergies or dietary restrictions more explicitly, despite the fact that the system has avoided using peanuts due to the allergy, some of the retrieved recipes contain peanuts which may indicate a need for better filtering.

Standard RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 7/10
Feedback: The Standard RAG System's output, while attempting to address the user's requirements, falls short in terms of relevance and accuracy. The vector search query, although similar to the user's

In [51]:
org_inp = inputs["input9"]
agentic_out = agentic_rag_outputs["input9"]
standard_out = standard_rag_outputs["input9"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 7/10
Feedback: The Agentic RAG System's vector search query is relevant to the task, and it retrieved various recipes that incorporate the provided ingredients. However, some of the retrieved recipes, such as "Mexican Hot Potatoes" and "Egg Foo Yong," do not directly relate to the task. The system's answer is mostly accurate in terms of providing a meal plan with nutritional information, but it includes a "Grilled Chicken and Vegetable Stir-Fry" recipe, which uses tilapia fillets instead of chicken and does not meet the daily protein target.

Standard RAG System:
- Context Relevance score: 9/10
- Answer Accuracy score: 9/10
Feedback: The Standard RAG System's vector search query is highly relevant to the task, but the retrieved recipes are less relevant to the task, such as "Taco Delight" and "Hamburger Barbecue." However, the system's answer is highly accurate in terms of providing a meal plan that meets the 

In [52]:
org_inp = inputs["input10"]
agentic_out = agentic_rag_outputs["input10"]
standard_out = standard_rag_outputs["input10"]
print(llm_judge(org_inp, agentic_out, standard_out))

Agentic RAG System:
- Context Relevance score: 6/10
- Answer Accuracy score: 7/10
Feedback: The Agentic RAG System's output is partially relevant to the user's input and task, as it provides a meal plan with recipes. However, the vector search query seems to be a simple list of ingredients, which may not be effective in retrieving relevant recipes. The retrieved recipes do not appear to be directly related to the user's ingredients or dietary goals. The meal plan meets some of the constraints, such as using provided ingredients, but the daily protein target is not closely met. 

Standard RAG System:
- Context Relevance score: 8/10
- Answer Accuracy score: 9/10
Feedback: The Standard RAG System's output is more relevant to the user's input and task, as it provides a comprehensive meal plan with recipes that incorporate the user's ingredients and meet the daily protein target. The vector search query is more specific and takes into account the user's dietary goals and restrictions. The r